In [3]:
  # 📌 Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 📌 Importar librerías necesarias
import pandas as pd
import numpy as np

# 📌 Definir la ruta del archivo guardado en el quinto paso
ruta_archivo = "/content/drive/MyDrive/Datos/5_Base_Imputadas_Cuentas.parquet"

# 📌 Cargar la base de datos
base_datos = pd.read_parquet(ruta_archivo)

# 📌 Mostrar información general de la base
print("🔎 Información general de la base de datos:")
print(base_datos.info())

# 📌 Vista previa de los primeros registros
print("\n🔎 Vista previa de la base de datos:")
print(base_datos.head())

# 🔹 **Crear una copia de la base para los indicadores**
base_indicadores = base_datos.copy()

# 📌 Ver información general de la base de datos
print("\n🔎 **Información general de la base de datos antes de calcular indicadores:**")
print(base_indicadores.info())

# 📌 Ver lista de columnas disponibles
print("\n📋 **Lista de columnas disponibles en la base de datos:**")
print(base_indicadores.columns)

# 📌 Mostrar los primeros registros para revisar valores
print("\n🔍 **Vista previa de los primeros registros:**")
print(base_indicadores.head())

# 📌 Verificar si las columnas clave existen antes de empezar los cálculos
columnas_necesarias = [
    'EBIT', 'DyA', 'AC', 'PCP', 'I', 'PN', 'AT', 'PT', 'PLP', 'GF', 'GA',
    'CPP', 'CPC', 'FCL', 'V'
]
faltantes = [col for col in columnas_necesarias if col not in base_indicadores.columns]

if faltantes:
    print(f"\n🚨 **Columnas faltantes antes de calcular indicadores:** {faltantes}")
else:
    print("\n✅ **Todas las columnas necesarias están presentes en la base de datos.**")

# 📌 Revisar tipos de datos de las columnas clave
print("\n📊 **Tipos de datos de las columnas clave:**")
print(base_indicadores[columnas_necesarias].dtypes)


# ==============================================================================
# 📌 SECCIÓN 1: CÁLCULO DE INDICADORES DE LIQUIDEZ
# ==============================================================================

# 🔹 **Cálculo de LC (Liquidez Corriente)**
# 📌 Fórmula: LC = AC / PCP
# 📌 Interpretación:
#    - Mide la capacidad de la empresa para cubrir sus pasivos corrientes con sus activos corrientes.
#    - Un valor alto sugiere buena liquidez, mientras que un valor bajo puede indicar problemas de solvencia a corto plazo.
#
# 📌 Manejo de casos:
# 1️⃣ **Si AC = 0 y PCP = 0 → NaN**
#    - ❓ **¿Qué significa?** Empresa sin activos ni pasivos corrientes. Puede ser una empresa inactiva o con datos inconsistentes.
#    - 🔹 **Manejo recomendado:**
#      - Mantener como **NaN** si se quiere revisar manualmente empresas sin estructura financiera clara.
#      - Imputar con **1** en modelos numéricos si se asume que la empresa tiene liquidez neutra sin operaciones.
#
# 2️⃣ **Si AC > 0 y PCP = 0 → ∞**
#    - Empresa sin pasivos corrientes pero con activos corrientes (totalmente líquida).
#
# 3️⃣ **Si PCP > 0 (sin importar AC) → Normal**
#    - Se calcula la división directamente, incluyendo el caso AC = 0 (resultado = 0).

base_indicadores['LC'] = np.where(
    (base_indicadores['AC'] == 0) & (base_indicadores['PCP'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['PCP'] == 0), np.inf,  # Caso 2: ∞ (cuando PCP = 0)
        base_indicadores['AC'] / base_indicadores['PCP']  # Caso 3 y 4 juntos
    )
)

# 🔹 **Resumen de los valores en LC**
casos_normales_LC = ((base_indicadores['PCP'] > 0)).sum()  # Casos normales incluyen AC = 0
casos_nan_LC = ((base_indicadores['AC'] == 0) & (base_indicadores['PCP'] == 0)).sum()
casos_inf_LC = ((base_indicadores['PCP'] == 0) & (base_indicadores['AC'] > 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en LC (Liquidez Corriente):**")
print(f"✅ Casos normales (incluye ceros): {casos_normales_LC}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_LC} → 📌 Puede representar empresas sin operaciones claras. Se recomienda revisar o imputar con 1 en modelos numéricos.")
print(f"♾️ Casos Infinito positivo: {casos_inf_LC}")

# 🔹 **Cálculo de PA (Prueba Ácida)**
# 📌 Fórmula: PA = (AC - I) / PCP
# 📌 Interpretación:
#    - Mide la capacidad de la empresa para cubrir sus pasivos corrientes **sin depender de sus inventarios**.
#    - Es una prueba más estricta de liquidez en comparación con la razón corriente (LC).
#
# 📌 Manejo de casos:
# 1️⃣ **Si (AC - I) = 0 y PCP = 0 → NaN**
#    - ❓ **¿Qué significa?** Empresa sin activos líquidos ni pasivos corrientes. Puede ser una empresa inactiva o con datos inconsistentes.
#    - 🔹 **Manejo recomendado:**
#      - Mantener como **NaN** si se quiere revisar empresas con estructura financiera indefinida.
#      - Imputar con **1** en modelos numéricos si se asume que la empresa tiene liquidez neutra sin operaciones.
#
# 2️⃣ **Si (AC - I) > 0 y PCP = 0 → ∞**
#    - Empresa sin pasivos corrientes pero con activos líquidos, es decir, **totalmente líquida**.
#
# 3️⃣ **Si PCP > 0 (sin importar AC - I) → Normal**
#    - Se calcula la división directamente, incluyendo el caso (AC - I) = 0 (resultado = 0).
#
# 4️⃣ **Si (AC - I) < 0 y PCP > 0 → Valor negativo**
#    - Indica que la empresa **tiene más inventarios que activos líquidos**, lo que puede ser un problema si necesita efectivo rápido.

base_indicadores['PA'] = np.where(
    ((base_indicadores['AC'] - base_indicadores['I']) == 0) & (base_indicadores['PCP'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['PCP'] == 0), np.inf,  # Caso 2: ∞ (cuando PCP = 0)
        (base_indicadores['AC'] - base_indicadores['I']) / base_indicadores['PCP']  # Casos 3, 4 y 5 juntos
    )
)

# 🔹 **Resumen de los valores en PA**
casos_normales_PA = ((base_indicadores['PCP'] > 0)).sum()  # Casos normales incluyen (AC - I) = 0 y negativos
casos_nan_PA = (((base_indicadores['AC'] - base_indicadores['I']) == 0) & (base_indicadores['PCP'] == 0)).sum()
casos_inf_PA = ((base_indicadores['PCP'] == 0) & ((base_indicadores['AC'] - base_indicadores['I']) > 0)).sum()
casos_negativos_PA = ((base_indicadores['PCP'] > 0) & ((base_indicadores['AC'] - base_indicadores['I']) < 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en PA (Prueba Ácida):**")
print(f"✅ Casos normales (incluye ceros y negativos): {casos_normales_PA}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_PA} → 📌 Puede representar empresas sin operaciones claras. Se recomienda revisar o imputar con 1 en modelos numéricos.")
print(f"♾️ Casos Infinito positivo: {casos_inf_PA}")
print(f"🔻 Casos negativos (más inventarios que activos líquidos): {casos_negativos_PA}")

# 🔹 **Cálculo de CTN (Capital de Trabajo Neto)**
# 📌 Fórmula: CTN = AC - PCP
# 📌 Interpretación:
#    - Representa la diferencia entre los **activos corrientes (AC)** y los **pasivos corrientes (PCP)**.
#    - Un valor positivo indica que la empresa tiene más activos líquidos que deudas a corto plazo.
#    - Un valor negativo sugiere que la empresa **puede enfrentar problemas de liquidez** para cubrir sus obligaciones inmediatas.
#
# 📌 Manejo de casos:
# 1️⃣ **Si AC y PCP son NaN → NaN**
#    - ❓ **¿Qué significa?** No hay datos disponibles para calcular el capital de trabajo neto.
#    - 🔹 **Manejo recomendado:**
#      - Mantener como **NaN** si se necesita identificar empresas con datos faltantes.
#      - Imputar con **0** si se asume que no hay diferencia entre activos y pasivos corrientes.
#
# 2️⃣ **Si AC está vacío y PCP tiene valor → CTN = -PCP**
#    - Empresa sin activos líquidos reportados, pero con pasivos corrientes. Indica un **capital de trabajo neto negativo**.
#
# 3️⃣ **Si PCP está vacío y AC tiene valor → CTN = AC**
#    - Empresa sin deudas de corto plazo, lo que implica **capital de trabajo neto igual a los activos corrientes**.
#
# 4️⃣ **Si ambos tienen valor → CTN = AC - PCP**
#    - Se realiza el cálculo normal.

base_indicadores['CTN'] = np.where(
    base_indicadores[['AC', 'PCP']].isna().all(axis=1), np.nan,  # Caso 1: NaN
    np.where(
        base_indicadores['AC'].isna(), -base_indicadores['PCP'],  # Caso 2: -PCP
        np.where(
            base_indicadores['PCP'].isna(), base_indicadores['AC'],  # Caso 3: AC
            base_indicadores['AC'] - base_indicadores['PCP']  # Caso 4: Normal
        )
    )
)

# 🔹 **Resumen de los valores en CTN**
casos_normales_CTN = ((~base_indicadores['CTN'].isna()) & (base_indicadores['CTN'] != np.inf)).sum()
casos_nan_CTN = base_indicadores['CTN'].isna().sum()
casos_negativos_CTN = (base_indicadores['CTN'] < 0).sum()
casos_positivos_CTN = (base_indicadores['CTN'] > 0).sum()
casos_cero_CTN = (base_indicadores['CTN'] == 0).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en CTN (Capital de Trabajo Neto):**")
print(f"✅ Casos normales (incluye positivos y negativos): {casos_normales_CTN}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_CTN} → 📌 Puede representar empresas con datos faltantes. Se recomienda imputar con 0 si no se quiere perder información en análisis.")
print(f"🔻 Casos negativos (pasivo corriente mayor a activos corrientes): {casos_negativos_CTN}")
print(f"✅ Casos positivos (empresa con activos corrientes superiores a pasivos corrientes): {casos_positivos_CTN}")
print(f"🔘 Casos en cero: {casos_cero_CTN}")



# 🔹 **Cálculo de LG (Liquidez General)**
# 📌 Fórmula: LG = (AC + AF) / PCP
# 📌 Interpretación:
#    - Representa la capacidad de la empresa para cubrir sus **pasivos corrientes (PCP)**
#      utilizando tanto **activos corrientes (AC)** como **activos fijos (AF)**.
#    - Se usa cuando los activos fijos pueden liquidarse para cubrir obligaciones de corto plazo.
#
# 📌 Manejo de casos:
# 1️⃣ **Si (AC + AF) = 0 y PCP = 0 → NaN**
#    - ❓ **¿Qué significa?** No hay activos ni pasivos registrados.
#    - 🔹 **Manejo recomendado:**
#      - Mantener como **NaN** si se necesita identificar empresas sin datos.
#      - Imputar con **1** si se quiere evitar divisiones problemáticas en análisis financieros.
#
# 2️⃣ **Si (AC + AF) > 0 y PCP = 0 → ∞**
#    - Empresa con activos pero sin pasivos de corto plazo, indicando **solvencia total**.
#
# 3️⃣ **Si (AC + AF) = 0 y PCP > 0 → 0**
#    - Empresa sin activos, lo que indica **incapacidad total para pagar deudas a corto plazo**.
#
# 4️⃣ **Si (AC + AF) > 0 y PCP > 0 → Normal**
#    - Se calcula la división.

base_indicadores['LG'] = np.where(
    ((base_indicadores['AC'] + base_indicadores['AF']) == 0) & (base_indicadores['PCP'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['PCP'] == 0), np.inf,  # Caso 2: ∞ (cuando PCP = 0)
        (base_indicadores['AC'] + base_indicadores['AF']) / base_indicadores['PCP']  # Casos 3 y 4 juntos
    )
)

# 🔹 **Resumen de los valores en LG**
casos_normales_LG = ((base_indicadores['PCP'] > 0)).sum()  # Casos normales incluyen (AC + AF) = 0
casos_nan_LG = (((base_indicadores['AC'] + base_indicadores['AF']) == 0) & (base_indicadores['PCP'] == 0)).sum()
casos_inf_LG = ((base_indicadores['PCP'] == 0) & ((base_indicadores['AC'] + base_indicadores['AF']) > 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en LG (Liquidez General):**")
print(f"✅ Casos normales (incluye ceros): {casos_normales_LG}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_LG} → 📌 Puede representar empresas sin datos. Se recomienda imputar con **1** si se quiere evitar problemas en cálculos financieros.")
print(f"♾️ Casos Infinito positivo: {casos_inf_LG}")


# ==============================================================================
# 📌 SECCIÓN 1: CÁLCULO DE INDICADORES DE LIQUIDEZ
# ==============================================================================

# 🔹 **Cálculo de T (Tesorería)**
# 📌 Fórmula: T = (AC - I) / PT
# 📌 Interpretación:
#    - Indica la proporción de los activos líquidos de la empresa (activos corrientes sin inventarios)
#      en relación con el pasivo total.
#    - Se usa para medir **cuánto de los pasivos totales pueden cubrirse sin depender de los inventarios**.
#
# 📌 Manejo de casos:
# 1️⃣ **Si (AC - I) = 0 y PT = 0 → NaN**
#    - ❓ **¿Qué significa?** No hay activos líquidos ni pasivos registrados.
#    - 🔹 **Manejo recomendado:**
#      - Mantener como **NaN** para análisis de calidad de datos.
#      - Imputar con **1** si se quiere evitar problemas en modelos numéricos.
#
# 2️⃣ **Si (AC - I) > 0 y PT = 0 → ∞**
#    - Empresa sin pasivos pero con activos líquidos, lo que sugiere **solvencia total**.
#
# 3️⃣ **Si (AC - I) = 0 y PT > 0 → 0**
#    - Empresa sin activos líquidos, lo que indica **fuerte dependencia de pasivos**.
#
# 4️⃣ **Si (AC - I) < 0 y PT > 0 → Negativo**
#    - Empresa con **más inventarios que activos líquidos**, indicando un **riesgo financiero alto**.
#
# 5️⃣ **Si (AC - I) > 0 y PT > 0 → Normal**
#    - Se calcula la división.

base_indicadores['T'] = np.where(
    ((base_indicadores['AC'] - base_indicadores['I']) == 0) & (base_indicadores['PT'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['PT'] == 0), np.inf,  # Caso 2: ∞ (cuando PT = 0)
        (base_indicadores['AC'] - base_indicadores['I']) / base_indicadores['PT']  # Casos 3, 4 y 5 juntos
    )
)

# 🔹 **Resumen de los valores en T**
casos_normales_T = ((base_indicadores['PT'] > 0)).sum()  # Casos normales incluyen (AC - I) = 0 y negativos
casos_nan_T = (((base_indicadores['AC'] - base_indicadores['I']) == 0) & (base_indicadores['PT'] == 0)).sum()
casos_inf_T = ((base_indicadores['PT'] == 0) & ((base_indicadores['AC'] - base_indicadores['I']) > 0)).sum()
casos_negativos_T = ((base_indicadores['PT'] > 0) & ((base_indicadores['AC'] - base_indicadores['I']) < 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en T (Tesorería):**")
print(f"✅ Casos normales (incluye ceros y negativos): {casos_normales_T}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_T} → 📌 Puede representar empresas sin datos. Se recomienda imputar con **1** si se quiere evitar problemas en cálculos financieros.")
print(f"♾️ Casos Infinito positivo: {casos_inf_T}")
print(f"🔻 Casos negativos (más inventarios que activos líquidos): {casos_negativos_T}")


# ==============================================================================
# 📌 SECCIÓN 1: CÁLCULO DE INDICADORES DE LIQUIDEZ
# ==============================================================================

# 🔹 **Cálculo de TN (Tesorería Neta)**
# 📌 Fórmula: TN = (AC - I - PCP) / PT
# 📌 Interpretación:
#    - Muestra cuánto de la tesorería líquida (activos corrientes sin inventarios menos pasivos corrientes)
#      puede cubrir los pasivos totales de la empresa.
#    - Un valor alto indica **capacidad de pago sólida**, mientras que un valor negativo sugiere **riesgo financiero**.
#
# 📌 Manejo de casos:
# 1️⃣ **Si (AC - I - PCP) = 0 y PT = 0 → NaN**
#    - ❓ **¿Qué significa?** No hay tesorería neta ni pasivos totales registrados.
#    - 🔹 **Manejo recomendado:**
#      - Mantener como **NaN** para revisión de datos.
#      - Imputar con **1** en modelos numéricos si se quiere evitar problemas de cálculo.
#
# 2️⃣ **Si (AC - I - PCP) > 0 y PT = 0 → ∞**
#    - Empresa sin pasivos totales pero con tesorería neta, lo que indica **máxima solvencia**.
#
# 3️⃣ **Si (AC - I - PCP) = 0 y PT > 0 → 0**
#    - La empresa tiene pasivos pero su tesorería neta es nula, lo que indica **equilibrio exacto**.
#
# 4️⃣ **Si (AC - I - PCP) < 0 y PT > 0 → Negativo**
#    - Empresa con **déficit de tesorería neta**, indicando **problemas de liquidez y riesgo financiero**.
#
# 5️⃣ **Si (AC - I - PCP) > 0 y PT > 0 → Normal**
#    - Se calcula la división.

base_indicadores['TN'] = np.where(
    ((base_indicadores['AC'] - base_indicadores['I'] - base_indicadores['PCP']) == 0) & (base_indicadores['PT'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['PT'] == 0), np.inf,  # Caso 2: ∞ (cuando PT = 0)
        (base_indicadores['AC'] - base_indicadores['I'] - base_indicadores['PCP']) / base_indicadores['PT']  # Casos 3, 4 y 5 juntos
    )
)

# 🔹 **Resumen de los valores en TN**
casos_normales_TN = ((base_indicadores['PT'] > 0)).sum()  # Casos normales incluyen valores positivos, negativos y cero
casos_nan_TN = (((base_indicadores['AC'] - base_indicadores['I'] - base_indicadores['PCP']) == 0) & (base_indicadores['PT'] == 0)).sum()
casos_inf_TN = ((base_indicadores['PT'] == 0) & ((base_indicadores['AC'] - base_indicadores['I'] - base_indicadores['PCP']) > 0)).sum()
casos_negativos_TN = ((base_indicadores['PT'] > 0) & ((base_indicadores['AC'] - base_indicadores['I'] - base_indicadores['PCP']) < 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en TN (Tesorería Neta):**")
print(f"✅ Casos normales (incluye positivos, negativos y ceros): {casos_normales_TN}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_TN} → 📌 Puede representar empresas sin datos. Se recomienda imputar con **1** en modelos numéricos.")
print(f"♾️ Casos Infinito positivo: {casos_inf_TN}")
print(f"🔻 Casos negativos (déficit de tesorería neta): {casos_negativos_TN}")



# ==============================================================================
# 📌 SECCIÓN 2: CÁLCULO DE INDICADORES DE ENDEUDAMIENTO Y COBERTURA
# ==============================================================================

# 🔹 **Cálculo de RET (Razón de Endeudamiento Total)**
# 📌 Fórmula: RET = PT / PN
# 📌 Interpretación:
#    - Muestra la proporción de los recursos de la empresa que proviene de deuda.
#    - Un valor alto indica una empresa altamente apalancada, lo que puede implicar
#      mayor riesgo financiero si la deuda es excesiva.
#
# 📌 Manejo de casos:
# 1️⃣ **Si PT = 0 y PN = 0 → NaN**
#    - ❓ **¿Qué significa?** Empresa sin patrimonio ni pasivos. Puede representar una empresa recién creada, en quiebra o con datos inconsistentes.
#    - 🔹 **Manejo recomendado:**
#      - Mantener como **NaN** si se quiere analizar manualmente casos de empresas sin estructura financiera.
#      - Imputar con **0** en modelos numéricos si se interpreta que una empresa sin deuda ni patrimonio no tiene riesgo financiero medible.
#
# 2️⃣ **Si PT > 0 y PN = 0 → ∞**
#    - Empresa con pasivos totales pero sin patrimonio (100% financiada con deuda).
#
# 3️⃣ **Si PT = 0 y PN > 0 → 0**
#    - Empresa sin deuda, financiada completamente con patrimonio.
#
# 4️⃣ **Si PT > 0 y PN > 0 → Normal**
#    - Se calcula la división directamente.

base_indicadores['RET'] = np.where(
    (base_indicadores['PT'] == 0) & (base_indicadores['PN'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['PN'] == 0), np.inf,  # Caso 2: ∞ (cuando PN = 0)
        base_indicadores['PT'] / base_indicadores['PN']  # Caso 3 y 4 juntos
    )
)

# 🔹 **Resumen de los valores en RET**
casos_normales_RET = ((base_indicadores['PN'] > 0)).sum()  # Casos normales incluyen PT = 0
casos_nan_RET = ((base_indicadores['PT'] == 0) & (base_indicadores['PN'] == 0)).sum()
casos_inf_RET = ((base_indicadores['PN'] == 0) & (base_indicadores['PT'] > 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en RET (Razón de Endeudamiento Total):**")
print(f"✅ Casos normales: {casos_normales_RET}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_RET} → 📌 Puede representar empresas sin estructura financiera clara. Se recomienda revisar o imputar con 0 en modelos numéricos.")
print(f"♾️ Casos Infinito positivo: {casos_inf_RET}")


# 🔹 **Cálculo de RECP (Razón de Endeudamiento Corriente con Patrimonio)**
# 📌 Fórmula: RECP = PCP / PN
# 📌 Interpretación:
#    - Mide la proporción del pasivo corriente (obligaciones a corto plazo) en relación con el patrimonio.
#    - Un valor alto sugiere que la empresa depende mucho del financiamiento a corto plazo.
#
# 📌 Manejo de casos:
# 1️⃣ **Si PCP = 0 y PN = 0 → NaN**
#    - ❓ **¿Qué significa?** Empresa sin patrimonio ni pasivos corrientes. Puede representar una empresa recién creada, inactiva o con datos incorrectos.
#    - 🔹 **Manejo recomendado:**
#      - Mantener como **NaN** si se quiere revisar manualmente la situación financiera.
#      - Imputar con **0** en modelos numéricos para evitar problemas de cálculo, ya que **una empresa sin patrimonio ni pasivos corrientes puede interpretarse como una empresa sin obligaciones de deuda en relación con su estructura financiera.**
#
# 2️⃣ **Si PCP > 0 y PN = 0 → ∞**
#    - Empresa con pasivos corrientes pero sin patrimonio (100% financiada con deuda a corto plazo).
#
# 3️⃣ **Si PCP = 0 y PN > 0 → 0**
#    - Empresa sin pasivos corrientes, totalmente financiada por patrimonio.
#
# 4️⃣ **Si PCP > 0 y PN > 0 → Normal**
#    - Se calcula la división directamente.

base_indicadores['RECP'] = np.where(
    (base_indicadores['PCP'] == 0) & (base_indicadores['PN'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['PN'] == 0), np.inf,  # Caso 2: ∞ (cuando PN = 0)
        base_indicadores['PCP'] / base_indicadores['PN']  # Caso 3 y 4 juntos
    )
)

# 🔹 **Resumen de los valores en RECP**
casos_normales_RECP = ((base_indicadores['PN'] > 0)).sum()  # Casos normales incluyen PCP = 0
casos_nan_RECP = ((base_indicadores['PCP'] == 0) & (base_indicadores['PN'] == 0)).sum()
casos_inf_RECP = ((base_indicadores['PN'] == 0) & (base_indicadores['PCP'] > 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en RECP (Razón de Endeudamiento Corriente con Patrimonio):**")
print(f"✅ Casos normales: {casos_normales_RECP}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_RECP} → 📌 Puede representar empresas sin estructura financiera clara. Se recomienda revisar o imputar con 0 en modelos numéricos.")
print(f"♾️ Casos Infinito positivo: {casos_inf_RECP}")


# ==============================================================================
# 📌 SECCIÓN 2: CÁLCULO DE INDICADORES DE ENDEUDAMIENTO Y COBERTURA DE DEUDA
# ==============================================================================

# 🔹 **Cálculo de RELP (Razón de Endeudamiento Largo Plazo)**
# 📌 Fórmula: RELP = PLP / PN
# 📌 Interpretación:
#    - Mide la proporción de la deuda a largo plazo en relación con el patrimonio.
#    - Un valor alto indica que la empresa **depende más de financiamiento a largo plazo**, lo que puede ser una estrategia
#      para reducir costos financieros a corto plazo, pero también aumenta el riesgo si la deuda es elevada.
#
# 📌 Manejo de casos:
# 1️⃣ **Si PLP = 0 y PN = 0 → NaN**
#    - ❓ **¿Qué significa?** Empresa sin pasivos de largo plazo ni patrimonio registrado.
#    - 🔹 **Manejo recomendado:**
#      - Mantener como **NaN** para análisis de calidad de datos.
#      - Imputar con **0** si se quiere evitar problemas en cálculos financieros.
#
# 2️⃣ **Si PLP > 0 y PN = 0 → ∞**
#    - Empresa con deuda a largo plazo pero sin patrimonio, lo que indica **dependencia total del endeudamiento**.
#
# 3️⃣ **Si PLP = 0 y PN > 0 → 0**
#    - Empresa sin pasivos a largo plazo, lo que sugiere **financiamiento totalmente con patrimonio**.
#
# 4️⃣ **Si PLP > 0 y PN > 0 → Normal**
#    - Se calcula la división.

base_indicadores['RELP'] = np.where(
    (base_indicadores['PLP'] == 0) & (base_indicadores['PN'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['PN'] == 0), np.inf,  # Caso 2: ∞ (cuando PN = 0)
        base_indicadores['PLP'] / base_indicadores['PN']  # Casos 3 y 4 juntos
    )
)

# 🔹 **Resumen de los valores en RELP**
casos_normales_RELP = ((base_indicadores['PN'] > 0)).sum()  # Casos normales incluyen PLP = 0
casos_nan_RELP = ((base_indicadores['PLP'] == 0) & (base_indicadores['PN'] == 0)).sum()
casos_inf_RELP = ((base_indicadores['PN'] == 0) & (base_indicadores['PLP'] > 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en RELP (Razón de Endeudamiento Largo Plazo):**")
print(f"✅ Casos normales: {casos_normales_RELP}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_RELP} → 📌 Puede representar empresas sin datos. Se recomienda imputar con **0** en modelos numéricos.")
print(f"♾️ Casos Infinito positivo: {casos_inf_RELP}")

# ==============================================================================
# 📌 SECCIÓN 2: CÁLCULO DE INDICADORES DE ENDEUDAMIENTO Y COBERTURA DE DEUDA
# ==============================================================================

# 🔹 **Cálculo de RAF (Razón de Apalancamiento Financiero)**
# 📌 Fórmula: RAF = AF / PN
# 📌 Interpretación:
#    - Mide qué porcentaje de los activos fijos está respaldado por el patrimonio de la empresa.
#    - Un valor alto indica que la empresa **ha invertido en activos fijos utilizando su propio capital**,
#      mientras que un valor bajo sugiere una mayor dependencia de la deuda para financiar estos activos.
#
# 📌 Manejo de casos:
# 1️⃣ **Si AF = 0 y PN = 0 → NaN**
#    - ❓ **¿Qué significa?** Empresa sin activos fijos y sin patrimonio registrado.
#    - 🔹 **Manejo recomendado:**
#      - Mantener como **NaN** para análisis de calidad de datos.
#      - Imputar con **0** en modelos numéricos si se quiere evitar errores en cálculos financieros.
#
# 2️⃣ **Si AF > 0 y PN = 0 → ∞**
#    - Empresa con activos fijos pero sin patrimonio, lo que indica **dependencia total de deuda o financiamiento externo**.
#
# 3️⃣ **Si AF = 0 y PN > 0 → 0**
#    - Empresa sin activos fijos, lo que sugiere **capital invertido en otros activos líquidos**.
#
# 4️⃣ **Si AF > 0 y PN > 0 → Normal**
#    - Se calcula la división.

base_indicadores['RAF'] = np.where(
    (base_indicadores['AF'] == 0) & (base_indicadores['PN'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['PN'] == 0), np.inf,  # Caso 2: ∞ (cuando PN = 0)
        base_indicadores['AF'] / base_indicadores['PN']  # Casos 3 y 4 juntos
    )
)

# 🔹 **Resumen de los valores en RAF**
casos_normales_RAF = ((base_indicadores['PN'] > 0)).sum()  # Casos normales incluyen AF = 0
casos_nan_RAF = ((base_indicadores['AF'] == 0) & (base_indicadores['PN'] == 0)).sum()
casos_inf_RAF = ((base_indicadores['PN'] == 0) & (base_indicadores['AF'] > 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en RAF (Razón de Apalancamiento Financiero):**")
print(f"✅ Casos normales: {casos_normales_RAF}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_RAF} → 📌 Puede representar empresas sin datos. Se recomienda imputar con **0** en modelos numéricos.")
print(f"♾️ Casos Infinito positivo: {casos_inf_RAF}")


# 🔹 **Cálculo de RAO (Rentabilidad de la Actividad Operativa)**
# 📌 Fórmula: RAO = EBIT / UO
# 📌 Interpretación:
#    - Mide la proporción de la utilidad operativa (UO) que proviene de la utilidad antes de impuestos e intereses (EBIT).
#    - Un valor alto indica **alta eficiencia en la actividad operativa**, mientras que un valor bajo sugiere que
#      los costos operativos pueden estar afectando significativamente la rentabilidad.
#
# 📌 Manejo de casos:
# 1️⃣ **Si EBIT = 0 y UO = 0 → NaN**
#    - ❓ **¿Qué significa?** Empresa sin actividad operativa registrada en ese periodo.
#    - 🔹 **Manejo recomendado:**
#      - **Imputar con 1** en modelos predictivos para evitar sesgos en la normalización.
#      - **Mantener como NaN** si el análisis busca evaluar periodos de inactividad.
#
# 2️⃣ **Si EBIT > 0 y UO = 0 → ∞**
#    - Empresa con utilidad antes de impuestos pero sin utilidad operativa registrada, lo que podría indicar
#      **costos operativos elevados o ingresos extraordinarios sin operaciones directas**.
#
# 3️⃣ **Si EBIT = 0 y UO > 0 → 0**
#    - Indica que la empresa tiene **utilidad operativa sin EBIT**, lo cual puede darse si los gastos financieros y
#      extraordinarios han reducido el EBIT a cero.
#
# 4️⃣ **Si EBIT > 0 y UO > 0 → Normal**
#    - Se calcula la división.

base_indicadores['RAO'] = np.where(
    (base_indicadores['EBIT'] == 0) & (base_indicadores['UO'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['UO'] == 0), np.inf,  # Caso 2: ∞ (cuando UO = 0)
        base_indicadores['EBIT'] / base_indicadores['UO']  # Casos 3 y 4 juntos
    )
)

# 🔹 **Resumen de los valores en RAO**
casos_normales_RAO = ((base_indicadores['UO'] > 0)).sum()  # Casos normales incluyen EBIT = 0
casos_nan_RAO = ((base_indicadores['EBIT'] == 0) & (base_indicadores['UO'] == 0)).sum()
casos_inf_RAO = ((base_indicadores['UO'] == 0) & (base_indicadores['EBIT'] > 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en RAO (Rentabilidad de la Actividad Operativa):**")
print(f"✅ Casos normales: {casos_normales_RAO}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_RAO} → 📌 Se recomienda **imputar con 1** para mantener la estabilidad en modelos predictivos.")
print(f"♾️ Casos Infinito positivo: {casos_inf_RAO}")



# 🔹 **Cálculo de RCD (Razón de Cobertura de Deuda Total)**
# 📌 Fórmula: RCD = FCL / PT
# 📌 Interpretación:
#    - Indica la capacidad de la empresa para **cubrir su deuda total (PT)** con su flujo de caja libre (FCL).
#    - Un valor alto sugiere que la empresa tiene suficiente flujo de caja para hacer frente a sus obligaciones.
#    - Un valor bajo indica que la empresa podría **tener dificultades en el pago de su deuda total**.
#
# 📌 Manejo de casos:
# 1️⃣ **Si FCL = 0 y PT = 0 → NaN**
#    - ❓ **¿Qué significa?** Empresa sin flujo de caja libre ni deuda total.
#    - 🔹 **Manejo recomendado:**
#      - **Mantener como NaN** si se busca analizar empresas sin apalancamiento.
#      - **Imputar con 1** en modelos predictivos, ya que la empresa no tiene deuda ni flujo de caja, por lo que
#        su cobertura de deuda sería neutral.
#
# 2️⃣ **Si FCL > 0 y PT = 0 → ∞**
#    - La empresa tiene flujo de caja libre pero **no tiene deuda total**, por lo que su capacidad de pago es infinita.
#
# 3️⃣ **Si FCL = 0 y PT > 0 → 0**
#    - La empresa tiene deuda total pero **no genera flujo de caja libre**, lo que puede ser una **señal de riesgo financiero**.
#
# 4️⃣ **Si FCL > 0 y PT > 0 → Normal**
#    - Se calcula la división.

base_indicadores['RCD'] = np.where(
    (base_indicadores['FCL'] == 0) & (base_indicadores['PT'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['PT'] == 0), np.inf,  # Caso 2: ∞ (cuando PT = 0)
        base_indicadores['FCL'] / base_indicadores['PT']  # Casos 3 y 4 juntos
    )
)

# 🔹 **Resumen de los valores en RCD**
casos_normales_RCD = ((base_indicadores['PT'] > 0)).sum()  # Casos normales incluyen FCL = 0
casos_nan_RCD = ((base_indicadores['FCL'] == 0) & (base_indicadores['PT'] == 0)).sum()
casos_inf_RCD = ((base_indicadores['PT'] == 0) & (base_indicadores['FCL'] > 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en RCD (Razón de Cobertura de Deuda Total):**")
print(f"✅ Casos normales: {casos_normales_RCD}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_RCD} → 📌 Se recomienda **imputar con 1** para modelos predictivos o mantener como NaN para análisis financiero.")
print(f"♾️ Casos Infinito positivo: {casos_inf_RCD}")

# 🔹 **Cálculo de RCID (Razón de Cobertura de Intereses con Deuda)**
# 📌 Fórmula: RCID = (EBIT + GF) / PT
# 📌 Interpretación:
#    - Mide la capacidad de la empresa para **cubrir su deuda total (PT)** con su utilidad antes de intereses e impuestos (EBIT) más los gastos financieros (GF).
#    - Un valor alto indica **suficiente capacidad de pago** de la deuda total.
#    - Un valor bajo sugiere que la empresa podría **tener dificultades en la cobertura de sus obligaciones**.
#
# 📌 Manejo de casos:
# 1️⃣ **Si (EBIT + GF) = 0 y PT = 0 → NaN**
#    - ❓ **¿Qué significa?** Empresa sin deuda y sin resultados operativos.
#    - 🔹 **Manejo recomendado:**
#      - **Mantener como NaN** en análisis financiero.
#      - **Imputar con 1** en modelos predictivos, reflejando una situación neutral.
#
# 2️⃣ **Si (EBIT + GF) > 0 y PT = 0 → ∞**
#    - La empresa tiene ganancias antes de intereses e impuestos, pero **no tiene deuda total**, por lo que la cobertura es infinita.
#
# 3️⃣ **Si (EBIT + GF) = 0 y PT > 0 → 0**
#    - La empresa tiene deuda total pero **no genera suficientes utilidades antes de intereses para cubrirla**, indicando un **riesgo financiero alto**.
#
# 4️⃣ **Si (EBIT + GF) > 0 y PT > 0 → Normal**
#    - Se calcula la división.

base_indicadores['RCID'] = np.where(
    ((base_indicadores['EBIT'] + base_indicadores['GF']) == 0) & (base_indicadores['PT'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['PT'] == 0), np.inf,  # Caso 2: ∞ (cuando PT = 0)
        (base_indicadores['EBIT'] + base_indicadores['GF']) / base_indicadores['PT']  # Casos 3 y 4 juntos
    )
)

# 🔹 **Resumen de los valores en RCID**
casos_normales_RCID = ((base_indicadores['PT'] > 0)).sum()  # Casos normales incluyen EBIT + GF = 0
casos_nan_RCID = (((base_indicadores['EBIT'] + base_indicadores['GF']) == 0) & (base_indicadores['PT'] == 0)).sum()
casos_inf_RCID = ((base_indicadores['PT'] == 0) & ((base_indicadores['EBIT'] + base_indicadores['GF']) > 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en RCID (Razón de Cobertura de Intereses con Deuda):**")
print(f"✅ Casos normales: {casos_normales_RCID}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_RCID} → 📌 Se recomienda **imputar con 1** en modelos predictivos o mantener como NaN para análisis financiero.")
print(f"♾️ Casos Infinito positivo: {casos_inf_RCID}")

# 🔹 **Cálculo de RCCP (Razón de Cobertura de Pasivos Corrientes con Flujo de Caja)**
# 📌 Fórmula: RCCP = FCL / PCP
# 📌 Interpretación:
#    - Indica **cuántas veces el flujo de caja libre (FCL) puede cubrir el pasivo corriente (PCP)**.
#    - Un valor alto sugiere que la empresa **tiene suficiente flujo de caja** para cubrir sus pasivos de corto plazo.
#    - Un valor bajo indica posible **riesgo de liquidez**.
#
# 📌 Manejo de casos:
# 1️⃣ **Si FCL = 0 y PCP = 0 → NaN**
#    - ❓ **¿Qué significa?** Empresa sin flujo de caja libre y sin pasivos corrientes.
#    - 🔹 **Manejo recomendado:**
#      - **Mantener como NaN** en análisis financiero.
#      - **Imputar con 1** en modelos predictivos para evitar sesgos.
#
# 2️⃣ **Si FCL > 0 y PCP = 0 → ∞**
#    - La empresa **no tiene pasivos corrientes**, pero sí flujo de caja, por lo que la cobertura es infinita.
#
# 3️⃣ **Si FCL = 0 y PCP > 0 → 0**
#    - La empresa **no genera flujo de caja libre** pero tiene pasivos corrientes.
#
# 4️⃣ **Si FCL > 0 y PCP > 0 → Normal**
#    - Se calcula la división.

base_indicadores['RCCP'] = np.where(
    (base_indicadores['FCL'] == 0) & (base_indicadores['PCP'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['PCP'] == 0), np.inf,  # Caso 2: ∞ (cuando PCP = 0)
        base_indicadores['FCL'] / base_indicadores['PCP']  # Casos 3 y 4 juntos
    )
)

# 🔹 **Resumen de los valores en RCCP**
casos_normales_RCCP = ((base_indicadores['PCP'] > 0)).sum()
casos_nan_RCCP = ((base_indicadores['FCL'] == 0) & (base_indicadores['PCP'] == 0)).sum()
casos_inf_RCCP = ((base_indicadores['PCP'] == 0) & (base_indicadores['FCL'] > 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en RCCP (Razón de Cobertura de Pasivos Corrientes con Flujo de Caja):**")
print(f"✅ Casos normales: {casos_normales_RCCP}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_RCCP} → 📌 Se recomienda **imputar con 1** en modelos predictivos o mantener como NaN en análisis financiero.")
print(f"♾️ Casos Infinito positivo: {casos_inf_RCCP}")
# 🔹 **Cálculo de RCLP (Razón de Cobertura de Pasivos No Corrientes con Flujo de Caja)**
# 📌 Fórmula: RCLP = FCL / PLP
# 📌 Interpretación:
#    - Mide cuántas veces el flujo de caja libre (FCL) puede cubrir el pasivo a largo plazo (PLP).
#    - Un valor alto indica **sólida capacidad de pago de deudas a largo plazo**.
#    - Un valor bajo sugiere posibles **dificultades en la sostenibilidad financiera**.
#
# 📌 Manejo de casos:
# 1️⃣ **Si FCL = 0 y PLP = 0 → NaN**
#    - ❓ **¿Qué significa?** Empresa sin flujo de caja libre y sin pasivos a largo plazo.
#    - 🔹 **Manejo recomendado:**
#      - **Mantener como NaN** en análisis financiero.
#      - **Imputar con 1** en modelos predictivos para evitar distorsiones.
#
# 2️⃣ **Si FCL > 0 y PLP = 0 → ∞**
#    - La empresa **no tiene pasivos a largo plazo**, pero sí flujo de caja, por lo que la cobertura es infinita.
#
# 3️⃣ **Si FCL = 0 y PLP > 0 → 0**
#    - La empresa tiene pasivos a largo plazo pero **no genera flujo de caja libre**.
#
# 4️⃣ **Si FCL > 0 y PLP > 0 → Normal**
#    - Se calcula la división.

base_indicadores['RCLP'] = np.where(
    (base_indicadores['FCL'] == 0) & (base_indicadores['PLP'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['PLP'] == 0), np.inf,  # Caso 2: ∞ (cuando PLP = 0)
        base_indicadores['FCL'] / base_indicadores['PLP']  # Casos 3 y 4 juntos
    )
)

# 🔹 **Resumen de los valores en RCLP**
casos_normales_RCLP = ((base_indicadores['PLP'] > 0)).sum()
casos_nan_RCLP = ((base_indicadores['FCL'] == 0) & (base_indicadores['PLP'] == 0)).sum()
casos_inf_RCLP = ((base_indicadores['PLP'] == 0) & (base_indicadores['FCL'] > 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en RCLP (Razón de Cobertura de Pasivos No Corrientes con Flujo de Caja):**")
print(f"✅ Casos normales: {casos_normales_RCLP}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_RCLP} → 📌 Se recomienda **imputar con 1** en modelos predictivos o mantener como NaN en análisis financiero.")
print(f"♾️ Casos Infinito positivo: {casos_inf_RCLP}")


# 🔹 **Cálculo de RCDT (Razón de Cobertura de Deuda Total con Flujo de Caja)**
# 📌 Fórmula: RCDT = FCL / PT
# 📌 Interpretación:
#    - Muestra cuántas veces el **flujo de caja libre (FCL)** puede cubrir la **deuda total (PT)**.
#    - Un valor alto indica **sólida capacidad de pago** de las deudas.
#    - Un valor bajo sugiere posibles **problemas de solvencia**.
#
# 📌 Manejo de casos:
# 1️⃣ **Si FCL = 0 y PT = 0 → NaN**
#    - ❓ **¿Qué significa?** Empresa sin flujo de caja libre y sin deuda total.
#    - 🔹 **Manejo recomendado:**
#      - **Mantener como NaN** en análisis financiero.
#      - **Imputar con 1** en modelos predictivos para evitar distorsiones.
#
# 2️⃣ **Si FCL > 0 y PT = 0 → ∞**
#    - La empresa **no tiene deuda total**, pero sí flujo de caja, por lo que la cobertura es infinita.
#
# 3️⃣ **Si FCL = 0 y PT > 0 → 0**
#    - La empresa tiene **deuda total pero no genera flujo de caja libre**, lo que puede ser **riesgoso**.
#
# 4️⃣ **Si FCL > 0 y PT > 0 → Normal**
#    - Se calcula la división.

base_indicadores['RCDT'] = np.where(
    (base_indicadores['FCL'] == 0) & (base_indicadores['PT'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['PT'] == 0), np.inf,  # Caso 2: ∞ (cuando PT = 0)
        base_indicadores['FCL'] / base_indicadores['PT']  # Casos 3 y 4 juntos
    )
)

# 🔹 **Resumen de los valores en RCDT**
casos_normales_RCDT = ((base_indicadores['PT'] > 0)).sum()
casos_nan_RCDT = ((base_indicadores['FCL'] == 0) & (base_indicadores['PT'] == 0)).sum()
casos_inf_RCDT = ((base_indicadores['PT'] == 0) & (base_indicadores['FCL'] > 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en RCDT (Razón de Cobertura de Deuda Total con Flujo de Caja):**")
print(f"✅ Casos normales: {casos_normales_RCDT}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_RCDT} → 📌 Se recomienda **imputar con 1** en modelos predictivos o mantener como NaN en análisis financiero.")
print(f"♾️ Casos Infinito positivo: {casos_inf_RCDT}")


# ==============================================================================
# 📌 SECCIÓN 3: Indicadores de Rentabilidad y Márgenes

# ==============================================================================



# 🔹 **Cálculo de EBITDA**
# 📌 Fórmula: EBITDA = EBIT + DyA
# 📌 Interpretación:
#    - Representa el beneficio de la empresa antes de intereses, impuestos, depreciación y amortización.
#    - Es una métrica clave para evaluar la rentabilidad operativa sin los efectos de decisiones de financiamiento o políticas fiscales.
#
# 📌 Manejo de casos:
# 1️⃣ Si EBIT y DyA son NaN → NaN (no se puede calcular)
# 2️⃣ Si EBIT es NaN y DyA tiene valor → EBITDA = DyA
# 3️⃣ Si DyA es NaN y EBIT tiene valor → EBITDA = EBIT
# 4️⃣ Si ambos tienen valores → EBITDA = EBIT + DyA (normal)

base_indicadores['EBITDA'] = np.where(
    base_indicadores[['EBIT', 'DyA']].isna().all(axis=1), np.nan,  # Caso 1: NaN
    np.where(
        base_indicadores['EBIT'].isna(), base_indicadores['DyA'],  # Caso 2: Solo DyA
        np.where(
            base_indicadores['DyA'].isna(), base_indicadores['EBIT'],  # Caso 3: Solo EBIT
            base_indicadores['EBIT'] + base_indicadores['DyA']  # Caso 4: Normal
        )
    )
)

# 🔹 **Resumen de los valores en EBITDA**
casos_normales_EBITDA = ((~base_indicadores['EBITDA'].isna())).sum()
casos_nan_EBITDA = base_indicadores['EBITDA'].isna().sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en EBITDA:**")
print(f"✅ Casos normales: {casos_normales_EBITDA}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_EBITDA} → 📌 Se recomienda **imputar con 0** en modelos predictivos o mantener como NaN en análisis financiero.")

# 🔹 **Cálculo de MB (Margen Bruto)**
# 📌 Fórmula: MB = UB / V
# 📌 Interpretación:
#    - Indica la rentabilidad bruta de la empresa antes de restar los gastos operacionales.
#    - Un margen bruto alto sugiere que la empresa tiene un fuerte control sobre sus costos de ventas.
#
# 📌 Manejo de casos:
# 1️⃣ Si UB = 0 y V = 0 → NaN (indeterminado, empresa sin ingresos ni utilidad bruta)
# 2️⃣ Si UB > 0 y V = 0 → ∞ (empresa con utilidad bruta pero sin ingresos reportados)
# 3️⃣ Si UB = 0 y V > 0 → 0 (empresa sin utilidad bruta, ingresos iguales a costos)
# 4️⃣ Si UB < 0 y V > 0 → Negativo (empresa con pérdidas brutas)
# 5️⃣ Si UB > 0 y V > 0 → Normal (se calcula la división)

base_indicadores['MB'] = np.where(
    (base_indicadores['UB'] == 0) & (base_indicadores['V'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['V'] == 0), np.inf,  # Caso 2: ∞ (cuando V = 0)
        base_indicadores['UB'] / base_indicadores['V']  # Casos 3, 4 y 5 juntos
    )
)

# 🔹 **Manejo recomendado:**
#    - Mantener como **NaN** para análisis detallado.
#    - Imputar con **0** si se considera que la empresa no tuvo actividad comercial.
#    - Considerar eliminar si el valor afecta el análisis de rentabilidad.

# 🔹 **Resumen de los valores en MB**
casos_normales_MB = ((base_indicadores['V'] > 0)).sum()
casos_nan_MB = ((base_indicadores['UB'] == 0) & (base_indicadores['V'] == 0)).sum()
casos_inf_MB = ((base_indicadores['V'] == 0) & (base_indicadores['UB'] > 0)).sum()
casos_negativos_MB = ((base_indicadores['UB'] < 0) & (base_indicadores['V'] > 0)).sum()

print("\n📊 **Distribución de casos en MB (Margen Bruto):**")
print(f"✅ Casos normales: {casos_normales_MB}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_MB} → 📌 Se recomienda **imputar con 0** en modelos predictivos, ya que implica que la empresa no tiene margen bruto.")

print(f"♾️ Casos Infinito positivo: {casos_inf_MB}")
print(f"🔻 Casos negativos (empresa con pérdidas brutas): {casos_negativos_MB}")


# 🔹 **Cálculo de MO (Margen Operacional)**
# 📌 Fórmula: MO = UO / V
# 📌 Interpretación:
#    - Representa la rentabilidad operativa después de restar los gastos administrativos y de ventas.
#    - Un margen operativo positivo indica que la empresa genera ganancias después de cubrir sus costos fijos y variables operacionales.
#
# 📌 Manejo de casos:
# 1️⃣ Si UO = 0 y V = 0 → NaN (indeterminado, empresa sin ingresos ni utilidad operacional)
# 2️⃣ Si UO > 0 y V = 0 → ∞ (empresa con utilidad operacional pero sin ingresos reportados)
# 3️⃣ Si UO = 0 y V > 0 → 0 (empresa sin utilidad operacional, ingresos igual a costos operacionales)
# 4️⃣ Si UO < 0 y V > 0 → Negativo (empresa con pérdidas operacionales)
# 5️⃣ Si UO > 0 y V > 0 → Normal (se calcula la división)

base_indicadores['MO'] = np.where(
    (base_indicadores['UO'] == 0) & (base_indicadores['V'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['V'] == 0), np.inf,  # Caso 2: ∞ (cuando V = 0)
        base_indicadores['UO'] / base_indicadores['V']  # Casos 3, 4 y 5 juntos
    )
)

# 🔹 **Manejo recomendado:**
#    - Mantener como **NaN** para análisis detallado.
#    - Imputar con **0** si se considera que la empresa no tuvo actividad comercial.
#    - Considerar eliminar si los valores NaN afectan la interpretación de la rentabilidad.

# 🔹 **Resumen de los valores en MO**
casos_normales_MO = ((base_indicadores['V'] > 0)).sum()
casos_nan_MO = ((base_indicadores['UO'] == 0) & (base_indicadores['V'] == 0)).sum()
casos_inf_MO = ((base_indicadores['V'] == 0) & (base_indicadores['UO'] > 0)).sum()
casos_negativos_MO = ((base_indicadores['UO'] < 0) & (base_indicadores['V'] > 0)).sum()

print("\n📊 **Distribución de casos en MO (Margen Operacional):**")
print(f"✅ Casos normales: {casos_normales_MO}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_MO} → 📌 Se recomienda **imputar con 0** en modelos predictivos, ya que implica que la empresa no tiene margen operacional.")
print(f"♾️ Casos Infinito positivo: {casos_inf_MO}")
print(f"🔻 Casos negativos (empresa con pérdidas operacionales): {casos_negativos_MO}")

# 🔹 **Cálculo de ROA (Rentabilidad sobre Activos)**
# 📌 Fórmula: ROA = UN / AT
# 📌 Interpretación:
#    - Mide la rentabilidad de la empresa en relación con sus activos totales.
#    - Un ROA alto indica una empresa eficiente en la generación de beneficios a partir de sus activos.
#
# 📌 Manejo de casos:
# 1️⃣ Si UN = 0 y AT = 0 → NaN (indeterminado, empresa sin activos ni utilidad neta)
# 2️⃣ Si UN > 0 y AT = 0 → ∞ (empresa con utilidad neta pero sin activos reportados)
# 3️⃣ Si UN = 0 y AT > 0 → 0 (empresa con activos pero sin utilidad neta)
# 4️⃣ Si UN < 0 y AT > 0 → Negativo (empresa con pérdidas netas, bajo desempeño)
# 5️⃣ Si UN > 0 y AT > 0 → Normal (se calcula la división)

base_indicadores['ROA'] = np.where(
    (base_indicadores['UN'] == 0) & (base_indicadores['AT'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['AT'] == 0), np.inf,  # Caso 2: ∞ (cuando AT = 0)
        base_indicadores['UN'] / base_indicadores['AT']  # Casos 3, 4 y 5 juntos
    )
)

# 🔹 **Manejo recomendado:**
#    - Mantener como **NaN** para revisión detallada.
#    - Imputar con **0** si se considera que la empresa no generó utilidad y tenía activos.
#    - **Eliminar registros NaN si afectan el análisis de rentabilidad.**

# 🔹 **Resumen de los valores en ROA**
casos_normales_ROA = ((base_indicadores['AT'] > 0)).sum()
casos_nan_ROA = ((base_indicadores['UN'] == 0) & (base_indicadores['AT'] == 0)).sum()
casos_inf_ROA = ((base_indicadores['AT'] == 0) & (base_indicadores['UN'] > 0)).sum()
casos_negativos_ROA = ((base_indicadores['UN'] < 0) & (base_indicadores['AT'] > 0)).sum()

print("\n📊 **Distribución de casos en ROA (Rentabilidad sobre Activos):**")
print(f"✅ Casos normales: {casos_normales_ROA}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_ROA} → 📌 Se recomienda **imputar con 0** en modelos predictivos, pues significa que la empresa no generó utilidad neta o no tenía activos reportados.")
print(f"♾️ Casos Infinito positivo: {casos_inf_ROA}")
print(f"🔻 Casos negativos (empresa con pérdidas): {casos_negativos_ROA}")


# 🔹 **Cálculo de ROE (Rentabilidad sobre Patrimonio)**
# 📌 Fórmula: ROE = UN / PN
# 📌 Interpretación:
#    - Mide la rentabilidad obtenida en relación con el patrimonio de los accionistas.
#    - Un ROE alto indica que la empresa está generando buenos rendimientos sobre el capital invertido.
#
# 📌 Manejo de casos:
# 1️⃣ Si UN = 0 y PN = 0 → NaN (indeterminado, empresa sin patrimonio ni utilidad neta)
# 2️⃣ Si UN > 0 y PN = 0 → ∞ (empresa con utilidad neta pero sin patrimonio)
# 3️⃣ Si UN = 0 y PN > 0 → 0 (empresa sin utilidad neta, solo con patrimonio)
# 4️⃣ Si UN < 0 y PN > 0 → Negativo (empresa con pérdidas, impacto en los accionistas)
# 5️⃣ Si UN > 0 y PN > 0 → Normal (se calcula la división)

base_indicadores['ROE'] = np.where(
    (base_indicadores['UN'] == 0) & (base_indicadores['PN'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['PN'] == 0), np.inf,  # Caso 2: ∞ (cuando PN = 0)
        base_indicadores['UN'] / base_indicadores['PN']  # Casos 3, 4 y 5 juntos
    )
)

# 🔹 **Manejo recomendado:**
#    - Mantener como **NaN** para revisión detallada.
#    - Imputar con **0** si se considera que la empresa no generó utilidad pero tenía patrimonio.
#    - **Eliminar registros NaN si afectan el análisis financiero.**

# 🔹 **Resumen de los valores en ROE**
casos_normales_ROE = ((base_indicadores['PN'] > 0)).sum()
casos_nan_ROE = ((base_indicadores['UN'] == 0) & (base_indicadores['PN'] == 0)).sum()
casos_inf_ROE = ((base_indicadores['PN'] == 0) & (base_indicadores['UN'] > 0)).sum()
casos_negativos_ROE = ((base_indicadores['UN'] < 0) & (base_indicadores['PN'] > 0)).sum()

print("\n📊 **Distribución de casos en ROE (Rentabilidad sobre Patrimonio):**")
print(f"✅ Casos normales: {casos_normales_ROE}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_ROE} → 📌 Se recomienda **mantener como NaN** si se quiere reflejar la falta de patrimonio, o **imputar con 0** en modelos predictivos para evitar errores.")
print(f"♾️ Casos Infinito positivo: {casos_inf_ROE}")
print(f"🔻 Casos negativos (empresa con pérdidas): {casos_negativos_ROE}")

# 🔹 **Cálculo de RSV (Rentabilidad sobre Ventas)**
# 📌 Fórmula: RSV = UN / V
# 📌 Interpretación:
#    - Indica qué porcentaje de cada unidad monetaria de ingresos se convierte en utilidad neta.
#    - Un RSV alto sugiere que la empresa es eficiente en la conversión de ingresos en beneficios.
#
# 📌 Manejo de casos:
# 1️⃣ Si UN = 0 y V = 0 → NaN (indeterminado, empresa sin ingresos ni utilidad)
# 2️⃣ Si UN > 0 y V = 0 → ∞ (empresa con utilidad neta pero sin ingresos reportados)
# 3️⃣ Si UN = 0 y V > 0 → 0 (empresa con ingresos pero sin utilidad neta)
# 4️⃣ Si UN < 0 y V > 0 → Negativo (empresa con pérdidas netas, bajo desempeño)
# 5️⃣ Si UN > 0 y V > 0 → Normal (se calcula la división)

base_indicadores['RSV'] = np.where(
    (base_indicadores['UN'] == 0) & (base_indicadores['V'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['V'] == 0), np.inf,  # Caso 2: ∞ (cuando V = 0)
        base_indicadores['UN'] / base_indicadores['V']  # Casos 3, 4 y 5 juntos
    )
)

# 🔹 **Manejo recomendado:**
#    - Mantener como **NaN** para revisión detallada.
#    - Imputar con **0** si se considera que la empresa no tuvo utilidad pero sí ventas.
#    - **Eliminar registros NaN si afectan el análisis de rentabilidad.**

# 🔹 **Resumen de los valores en RSV**
casos_normales_RSV = ((base_indicadores['V'] > 0)).sum()
casos_nan_RSV = ((base_indicadores['UN'] == 0) & (base_indicadores['V'] == 0)).sum()
casos_inf_RSV = ((base_indicadores['V'] == 0) & (base_indicadores['UN'] > 0)).sum()
casos_negativos_RSV = ((base_indicadores['UN'] < 0) & (base_indicadores['V'] > 0)).sum()

print("\n📊 **Distribución de casos en RSV (Rentabilidad sobre Ventas):**")
print(f"✅ Casos normales: {casos_normales_RSV}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_RSV} → 📌 Se recomienda **imputar con 0** en modelos predictivos, ya que indica que la empresa no tuvo rentabilidad sobre ventas.")
print(f"♾️ Casos Infinito positivo: {casos_inf_RSV}")
print(f"🔻 Casos negativos (empresa con pérdidas): {casos_negativos_RSV}")



# 🔹 **Cálculo de ROI (Retorno sobre la Inversión)**
# 📌 Fórmula: ROI = (UN + GF + Imp) / AF
# 📌 Interpretación:
#    - Mide la rentabilidad generada en relación con los activos fijos de la empresa.
#    - Un ROI alto indica que la empresa es eficiente en el uso de sus activos fijos.
#
# 📌 Manejo de casos:
# 1️⃣ Si (UN + GF + Imp) = 0 y AF = 0 → NaN (indeterminado, empresa sin inversión ni rentabilidad)
# 2️⃣ Si (UN + GF + Imp) > 0 y AF = 0 → ∞ (empresa con rentabilidad pero sin inversión reportada)
# 3️⃣ Si (UN + GF + Imp) = 0 y AF > 0 → 0 (empresa con inversión pero sin rentabilidad)
# 4️⃣ Si (UN + GF + Imp) < 0 y AF > 0 → Negativo (empresa con pérdidas netas en relación con la inversión)
# 5️⃣ Si (UN + GF + Imp) > 0 y AF > 0 → Normal (se calcula la división)

base_indicadores['ROI'] = np.where(
    ((base_indicadores['UN'] + base_indicadores['GF'] + base_indicadores['Imp']) == 0) & (base_indicadores['AF'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['AF'] == 0), np.inf,  # Caso 2: ∞ (cuando AF = 0)
        (base_indicadores['UN'] + base_indicadores['GF'] + base_indicadores['Imp']) / base_indicadores['AF']  # Casos 3, 4 y 5 juntos
    )
)

# 🔹 **Manejo recomendado:**
#    - Mantener como **NaN** para revisión detallada.
#    - Imputar con **0** si se considera que la empresa no tuvo rentabilidad pero sí inversión.
#    - **Eliminar registros NaN si afectan el análisis financiero.**

# 🔹 **Resumen de los valores en ROI**
casos_normales_ROI = ((base_indicadores['AF'] > 0)).sum()
casos_nan_ROI = (((base_indicadores['UN'] + base_indicadores['GF'] + base_indicadores['Imp']) == 0) & (base_indicadores['AF'] == 0)).sum()
casos_inf_ROI = ((base_indicadores['AF'] == 0) & ((base_indicadores['UN'] + base_indicadores['GF'] + base_indicadores['Imp']) > 0)).sum()
casos_negativos_ROI = (((base_indicadores['UN'] + base_indicadores['GF'] + base_indicadores['Imp']) < 0) & (base_indicadores['AF'] > 0)).sum()

print("\n📊 **Distribución de casos en ROI (Retorno sobre la Inversión):**")
print(f"✅ Casos normales: {casos_normales_ROI}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_ROI} → 📌 Se recomienda **mantener como NaN** si se quiere reflejar la falta de inversión, o **imputar con 0** en modelos predictivos para evitar errores.")
print(f"♾️ Casos Infinito positivo: {casos_inf_ROI}")
print(f"🔻 Casos negativos (empresa con pérdidas en relación con la inversión): {casos_negativos_ROI}")


# 🔹 **Cálculo de ME (Margen EBITDA)**
# 📌 Fórmula: ME = EBITDA / V
# 📌 Interpretación:
#    - Indica qué porcentaje de los ingresos de la empresa se convierte en EBITDA.
#    - Un ME alto sugiere que la empresa tiene un modelo de negocio eficiente con menores costos operativos.
#    - Es un indicador clave en la valoración de empresas y su capacidad de generación de flujo de caja.
#
# 📌 Manejo de casos:
# 1️⃣ Si EBITDA = 0 y V = 0 → NaN (indeterminado, empresa sin ingresos ni EBITDA)
# 2️⃣ Si EBITDA > 0 y V = 0 → ∞ (empresa con EBITDA positivo pero sin ingresos reportados)
# 3️⃣ Si EBITDA = 0 y V > 0 → 0 (empresa con ingresos pero sin EBITDA)
# 4️⃣ Si EBITDA < 0 y V > 0 → Negativo (empresa con pérdidas operativas)
# 5️⃣ Si EBITDA > 0 y V > 0 → Normal (se calcula la división)

base_indicadores['ME'] = np.where(
    (base_indicadores['EBITDA'] == 0) & (base_indicadores['V'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['V'] == 0), np.inf,  # Caso 2: ∞ (cuando V = 0)
        base_indicadores['EBITDA'] / base_indicadores['V']  # Casos 3, 4 y 5 juntos
    )
)

# 🔹 **Manejo recomendado:**
#    - Mantener como **NaN** para revisión detallada.
#    - Imputar con **0** si se considera que la empresa no generó EBITDA pero sí tuvo ingresos.
#    - **Eliminar registros NaN si afectan la interpretación de la rentabilidad operativa.**

# 🔹 **Resumen de los valores en ME**
casos_normales_ME = ((base_indicadores['V'] > 0)).sum()
casos_nan_ME = ((base_indicadores['EBITDA'] == 0) & (base_indicadores['V'] == 0)).sum()
casos_inf_ME = ((base_indicadores['V'] == 0) & (base_indicadores['EBITDA'] > 0)).sum()
casos_negativos_ME = ((base_indicadores['EBITDA'] < 0) & (base_indicadores['V'] > 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en ME (Margen EBITDA):**")
print(f"✅ Casos normales: {casos_normales_ME}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_ME} → 📌 Se recomienda **imputar con 0** en modelos predictivos, pues indica que la empresa no generó EBITDA.")
print(f"♾️ Casos Infinito positivo: {casos_inf_ME}")
print(f"🔻 Casos negativos (empresa con EBITDA negativo): {casos_negativos_ME}")




# ==============================================================================
# 📌 SECCIÓN 4: CÁLCULO DE Indicadores de Rotación y Eficiencia
# ==============================================================================


# 🔹 **Cálculo de RA (Rotación de Activos)**
# 📌 Fórmula: RA = V / AT
# 📌 Interpretación:
#    - Indica la eficiencia con la que una empresa utiliza sus activos para generar ingresos.
#    - Un valor más alto sugiere una mejor utilización de los activos para generar ventas.
#
# 📌 Manejo de casos:
# 1️⃣ Si V = 0 y AT = 0 → NaN (indeterminado, empresa sin ingresos ni activos)
# 2️⃣ Si V > 0 y AT = 0 → ∞ (empresa con ventas pero sin activos reportados)
# 3️⃣ Si V = 0 y AT > 0 → 0 (empresa con activos pero sin ingresos)
# 4️⃣ Si V > 0 y AT > 0 → Normal (se calcula la división)

base_indicadores['RA'] = np.where(
    (base_indicadores['V'] == 0) & (base_indicadores['AT'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['AT'] == 0), np.inf,  # Caso 2: ∞ (cuando AT = 0)
        base_indicadores['V'] / base_indicadores['AT']  # Caso 3 y 4 juntos
    )
)

# 🔹 **Resumen de los valores en RA**
casos_normales_RA = ((base_indicadores['AT'] > 0)).sum()  # Casos normales incluyen V=0
casos_nan_RA = ((base_indicadores['V'] == 0) & (base_indicadores['AT'] == 0)).sum()
casos_inf_RA = ((base_indicadores['AT'] == 0) & (base_indicadores['V'] > 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en RA (Rotación de Activos):**")
print(f"✅ Casos normales: {casos_normales_RA}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_RA} → 📌 Se recomienda **imputar con 0** en modelos predictivos o mantener como NaN en análisis financiero.")
print(f"♾️ Casos Infinito positivo: {casos_inf_RA} → 📌 Se recomienda **mantener como ∞**, pues indica ventas sin activos reportados.")

# 🔹 **Cálculo de RI (Rotación de Inventarios)**
# 📌 Fórmula: RI = V / I
# 📌 Interpretación:
#    - Indica cuántas veces la empresa vende y reemplaza su inventario en un período determinado.
#    - Un valor alto sugiere que el inventario se mueve rápido, mientras que un valor bajo puede indicar acumulación.
#
# 📌 Manejo de casos:
# 1️⃣ Si V = 0 y I = 0 → NaN (indeterminado, empresa sin ventas ni inventarios)
# 2️⃣ Si V > 0 y I = 0 → ∞ (empresa con ventas pero sin inventario reportado)
# 3️⃣ Si V = 0 y I > 0 → 0 (empresa con inventario pero sin ventas)
# 4️⃣ Si V > 0 y I > 0 → Normal (se calcula la división)

base_indicadores['RI'] = np.where(
    (base_indicadores['V'] == 0) & (base_indicadores['I'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['I'] == 0), np.inf,  # Caso 2: ∞ (cuando I = 0)
        base_indicadores['V'] / base_indicadores['I']  # Caso 3 y 4 juntos
    )
)

# 🔹 **Resumen de los valores en RI**
casos_normales_RI = ((base_indicadores['I'] > 0)).sum()  # Casos normales incluyen V=0
casos_nan_RI = ((base_indicadores['V'] == 0) & (base_indicadores['I'] == 0)).sum()
casos_inf_RI = ((base_indicadores['I'] == 0) & (base_indicadores['V'] > 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en RI (Rotación de Inventarios):**")
print(f"✅ Casos normales: {casos_normales_RI}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_RI} → 📌 Se recomienda **imputar con 0** en modelos predictivos o mantener como NaN en análisis financiero.")
print(f"♾️ Casos Infinito positivo: {casos_inf_RI} → 📌 Se recomienda **mantener como ∞**, pues indica ventas sin inventario registrado.")

# 🔹 **Cálculo de RCC (Rotación de Cuentas por Cobrar)**
# 📌 Fórmula: RCC = V / CPC
# 📌 Interpretación:
#    - Indica cuántas veces la empresa convierte sus cuentas por cobrar en efectivo durante un período.
#    - Un valor alto sugiere que la empresa cobra rápidamente a sus clientes, mientras que un valor bajo puede indicar problemas de liquidez.
#
# 📌 Manejo de casos:
# 1️⃣ Si V = 0 y CPC = 0 → NaN (indeterminado, empresa sin ventas ni cuentas por cobrar)
# 2️⃣ Si V > 0 y CPC = 0 → ∞ (empresa con ventas pero sin cuentas por cobrar registradas)
# 3️⃣ Si V = 0 y CPC > 0 → 0 (empresa con cuentas por cobrar pero sin ventas)
# 4️⃣ Si V > 0 y CPC > 0 → Normal (se calcula la división)

base_indicadores['RCC'] = np.where(
    (base_indicadores['V'] == 0) & (base_indicadores['CPC'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['CPC'] == 0), np.inf,  # Caso 2: ∞ (cuando CPC = 0)
        base_indicadores['V'] / base_indicadores['CPC']  # Caso 3 y 4 juntos
    )
)

# 🔹 **Resumen de los valores en RCC**
casos_normales_RCC = ((base_indicadores['CPC'] > 0)).sum()  # Casos normales incluyen V=0
casos_nan_RCC = ((base_indicadores['V'] == 0) & (base_indicadores['CPC'] == 0)).sum()
casos_inf_RCC = ((base_indicadores['CPC'] == 0) & (base_indicadores['V'] > 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en RCC (Rotación de Cuentas por Cobrar):**")
print(f"✅ Casos normales: {casos_normales_RCC}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_RCC} → 📌 Se recomienda **imputar con 0** en modelos predictivos o mantener como NaN en análisis financiero.")
print(f"♾️ Casos Infinito positivo: {casos_inf_RCC} → 📌 Se recomienda **mantener como ∞**, pues indica ventas sin cuentas por cobrar registradas.")


# 🔹 **Cálculo de RCP (Rotación de Cuentas por Pagar)**
# 📌 Fórmula: RCP = V / CPP
# 📌 Interpretación:
#    - Indica cuántas veces la empresa paga sus cuentas a proveedores en un período determinado.
#    - Un valor alto sugiere que la empresa paga rápidamente a sus proveedores, mientras que un valor bajo puede indicar dependencia del crédito.
#
# 📌 Manejo de casos:
# 1️⃣ Si V = 0 y CPP = 0 → NaN (indeterminado, empresa sin ventas ni cuentas por pagar)
# 2️⃣ Si V > 0 y CPP = 0 → ∞ (empresa con ventas pero sin cuentas por pagar registradas)
# 3️⃣ Si V = 0 y CPP > 0 → 0 (empresa con cuentas por pagar pero sin ventas)
# 4️⃣ Si V > 0 y CPP > 0 → Normal (se calcula la división)

base_indicadores['RCP'] = np.where(
    (base_indicadores['V'] == 0) & (base_indicadores['CPP'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['CPP'] == 0), np.inf,  # Caso 2: ∞ (cuando CPP = 0)
        base_indicadores['V'] / base_indicadores['CPP']  # Caso 3 y 4 juntos
    )
)

# 🔹 **Resumen de los valores en RCP**
casos_normales_RCP = ((base_indicadores['CPP'] > 0)).sum()  # Casos normales incluyen V=0
casos_nan_RCP = ((base_indicadores['V'] == 0) & (base_indicadores['CPP'] == 0)).sum()
casos_inf_RCP = ((base_indicadores['CPP'] == 0) & (base_indicadores['V'] > 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en RCP (Rotación de Cuentas por Pagar):**")
print(f"✅ Casos normales: {casos_normales_RCP}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_RCP} → 📌 Se recomienda **imputar con 0** en modelos predictivos o mantener como NaN en análisis financiero.")
print(f"♾️ Casos Infinito positivo: {casos_inf_RCP} → 📌 Se recomienda **mantener como ∞**, pues indica ventas sin cuentas por pagar registradas.")
# 🔹 **Cálculo de PPC (Período Promedio de Cobro)**
# 📌 Fórmula: PPC = CPC / (V / 365)
# 📌 Interpretación:
#    - Indica cuántos días, en promedio, tarda la empresa en cobrar sus cuentas por cobrar.
#    - Un valor alto sugiere que la empresa tarda en recibir el dinero de sus clientes.
#
# 📌 Manejo de casos:
# 1️⃣ Si CPC = 0 y V = 0 → NaN (indeterminado, empresa sin cuentas por cobrar ni ventas)
# 2️⃣ Si CPC > 0 y V = 0 → ∞ (empresa con cuentas por cobrar pero sin ventas registradas)
# 3️⃣ Si CPC = 0 y V > 0 → 0 (empresa sin cuentas por cobrar, solo ventas de contado)
# 4️⃣ Si CPC > 0 y V > 0 → Normal (se calcula la división)

base_indicadores['PPC'] = np.where(
    (base_indicadores['CPC'] == 0) & (base_indicadores['V'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['V'] == 0), np.inf,  # Caso 2: ∞ (cuando V = 0)
        base_indicadores['CPC'] / (base_indicadores['V'] / 365)  # Caso 3 y 4 juntos
    )
)

# 🔹 **Resumen de los valores en PPC**
casos_normales_PPC = ((base_indicadores['CPC'] > 0)).sum()  # Casos normales incluyen V=0
casos_nan_PPC = ((base_indicadores['CPC'] == 0) & (base_indicadores['V'] == 0)).sum()
casos_inf_PPC = ((base_indicadores['V'] == 0) & (base_indicadores['CPC'] > 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en PPC (Período Promedio de Cobro):**")
print(f"✅ Casos normales: {casos_normales_PPC}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_PPC} → 📌 Se recomienda **imputar con 0** en modelos predictivos o mantener como NaN en análisis financiero.")
print(f"♾️ Casos Infinito positivo: {casos_inf_PPC} → 📌 Se recomienda **mantener como ∞**, pues indica cuentas por cobrar sin ventas registradas.")

# 🔹 **Cálculo de PPP (Período Promedio de Pago)**
# 📌 Fórmula: PPP = CPP / (V / 365)
# 📌 Interpretación:
#    - Indica cuántos días, en promedio, tarda la empresa en pagar sus cuentas por pagar.
#    - Un valor alto sugiere que la empresa está retrasando sus pagos, lo cual puede ser indicativo de problemas de liquidez.
#
# 📌 Manejo de casos:
# 1️⃣ Si CPP = 0 y V = 0 → NaN (indeterminado, empresa sin cuentas por pagar ni ventas)
# 2️⃣ Si CPP > 0 y V = 0 → ∞ (empresa con cuentas por pagar pero sin ventas registradas)
# 3️⃣ Si CPP = 0 y V > 0 → 0 (empresa sin cuentas por pagar, solo ventas de contado)
# 4️⃣ Si CPP > 0 y V > 0 → Normal (se calcula la división)

base_indicadores['PPP'] = np.where(
    (base_indicadores['CPP'] == 0) & (base_indicadores['V'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['V'] == 0), np.inf,  # Caso 2: ∞ (cuando V = 0)
        base_indicadores['CPP'] / (base_indicadores['V'] / 365)  # Caso 3 y 4 juntos
    )
)

# 🔹 **Resumen de los valores en PPP**
casos_normales_PPP = ((base_indicadores['CPP'] > 0)).sum()  # Casos normales incluyen V=0
casos_nan_PPP = ((base_indicadores['CPP'] == 0) & (base_indicadores['V'] == 0)).sum()
casos_inf_PPP = ((base_indicadores['V'] == 0) & (base_indicadores['CPP'] > 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en PPP (Período Promedio de Pago):**")
print(f"✅ Casos normales: {casos_normales_PPP}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_PPP} → 📌 Se recomienda **imputar con 0** en modelos predictivos o mantener como NaN en análisis financiero.")
print(f"♾️ Casos Infinito positivo: {casos_inf_PPP} → 📌 Se recomienda **mantener como ∞**, pues indica cuentas por pagar sin ventas registradas.")
# 🔹 **Cálculo de PPI (Período Promedio de Inventarios)**
# 📌 Fórmula: PPI = I / (V / 365)
# 📌 Interpretación:
#    - Indica cuántos días, en promedio, la empresa tarda en vender su inventario.
#    - Un valor alto sugiere que la empresa tiene problemas para rotar su inventario, lo cual puede afectar su liquidez.
#
# 📌 Manejo de casos:
# 1️⃣ Si I = 0 y V = 0 → NaN (indeterminado, empresa sin inventario ni ventas)
# 2️⃣ Si I > 0 y V = 0 → ∞ (empresa con inventario pero sin ventas registradas)
# 3️⃣ Si I = 0 y V > 0 → 0 (empresa sin inventario, solo ventas de servicios o productos de terceros)
# 4️⃣ Si I > 0 y V > 0 → Normal (se calcula la división)

base_indicadores['PPI'] = np.where(
    (base_indicadores['I'] == 0) & (base_indicadores['V'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['V'] == 0), np.inf,  # Caso 2: ∞ (cuando V = 0)
        base_indicadores['I'] / (base_indicadores['V'] / 365)  # Caso 3 y 4 juntos
    )
)

# 🔹 **Resumen de los valores en PPI**
casos_normales_PPI = ((base_indicadores['I'] > 0)).sum()  # Casos normales incluyen V=0
casos_nan_PPI = ((base_indicadores['I'] == 0) & (base_indicadores['V'] == 0)).sum()
casos_inf_PPI = ((base_indicadores['V'] == 0) & (base_indicadores['I'] > 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en PPI (Período Promedio de Inventarios):**")
print(f"✅ Casos normales: {casos_normales_PPI}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_PPI} → 📌 Se recomienda **imputar con 0** en modelos predictivos o mantener como NaN en análisis financiero.")
print(f"♾️ Casos Infinito positivo: {casos_inf_PPI} → 📌 Se recomienda **mantener como ∞**, pues indica inventario sin ventas registradas.")


# ==============================================================================
# 📌 SECCIÓN 5: CÁLCULO DE Indicadores de Productividad y Generación de Valor
# ==============================================================================

# 🔹 **Cálculo de KTNO (Capital de Trabajo Neto Operativo)**
# 📌 Fórmula: KTNO = CPC + I - CPP
# 📌 Interpretación:
#    - Representa la liquidez operativa de la empresa considerando cuentas por cobrar, inventarios y cuentas por pagar.
#    - Un valor alto indica que la empresa tiene más recursos disponibles a corto plazo, mientras que un valor negativo sugiere problemas de liquidez.
#
# 📌 Manejo de casos:
# 1️⃣ Si CPC, I y CPP son NaN → NaN (no se puede calcular)
# 2️⃣ Si solo CPC e I son NaN, pero CPP tiene valor → KTNO = -CPP
# 3️⃣ Si solo CPP es NaN, pero CPC e I tienen valores → KTNO = CPC + I
# 4️⃣ Si hay valores en los tres → KTNO = CPC + I - CPP (normal)

base_indicadores['KTNO'] = np.where(
    base_indicadores[['CPC', 'I', 'CPP']].isna().all(axis=1), np.nan,  # Caso 1: NaN
    np.where(
        base_indicadores[['CPC', 'I']].isna().all(axis=1), -base_indicadores['CPP'],  # Caso 2: Solo CPP
        np.where(
            base_indicadores['CPP'].isna(), base_indicadores['CPC'] + base_indicadores['I'],  # Caso 3: Sin CPP
            base_indicadores['CPC'] + base_indicadores['I'] - base_indicadores['CPP']  # Caso 4: Normal
        )
    )
)

# 🔹 **Resumen de los valores en KTNO**
casos_normales_KTNO = ((~base_indicadores['KTNO'].isna())).sum()
casos_nan_KTNO = base_indicadores['KTNO'].isna().sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en KTNO (Capital de Trabajo Neto Operativo):**")
print(f"✅ Casos normales: {casos_normales_KTNO}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_KTNO} → 📌 Se recomienda **imputar con 0** en modelos predictivos o mantener como NaN en análisis financiero.")
# 🔹 **Cálculo de PKT (Productividad del Capital de Trabajo)**
# 📌 Fórmula: PKT = V / KTNO
# 📌 Interpretación:
#    - Mide cuántas veces las ventas cubren el capital de trabajo neto operativo.
#    - Un valor alto sugiere que la empresa genera muchas ventas con su capital de trabajo.
#    - Un valor bajo indica que la empresa necesita más capital de trabajo para generar ventas.
#
# 📌 Manejo de casos:
# 1️⃣ Si V = 0 y KTNO = 0 → NaN (indeterminado)
# 2️⃣ Si V > 0 y KTNO = 0 → ∞ (empresa con ingresos pero sin capital de trabajo)
# 3️⃣ Si V = 0 y KTNO > 0 → 0 (empresa sin ingresos, pero con capital de trabajo)
# 4️⃣ Si V > 0 y KTNO > 0 → Normal (se calcula la división)

base_indicadores['PKT'] = np.where(
    (base_indicadores['V'] == 0) & (base_indicadores['KTNO'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['KTNO'] == 0), np.inf,  # Caso 2: ∞ (cuando KTNO = 0)
        base_indicadores['V'] / base_indicadores['KTNO']  # Caso 3 y 4 juntos
    )
)

# 🔹 **Resumen de los valores en PKT**
casos_normales_PKT = ((base_indicadores['KTNO'] > 0)).sum()  # Casos normales incluyen V = 0
casos_nan_PKT = ((base_indicadores['V'] == 0) & (base_indicadores['KTNO'] == 0)).sum()
casos_inf_PKT = ((base_indicadores['KTNO'] == 0) & (base_indicadores['V'] > 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en PKT (Productividad del Capital de Trabajo):**")
print(f"✅ Casos normales: {casos_normales_PKT}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_PKT} → 📌 Se recomienda **imputar con 1** en modelos predictivos o mantener como NaN en análisis financiero.")
print(f"♾️ Casos Infinito positivo: {casos_inf_PKT}")
# 🔹 **Cálculo de PCD (Palanca de Crecimiento)**
# 📌 Fórmula: PCD = ME / PKT
# 📌 Interpretación:
#    - Mide qué tan eficiente es la empresa en transformar su margen EBITDA en crecimiento.
#    - Un valor alto sugiere que la empresa genera altos márgenes con una baja inversión en capital de trabajo.
#
# 📌 Manejo de casos:
# 1️⃣ Si ME = 0 y PKT = 0 → NaN (indeterminado, no hay margen ni productividad)
# 2️⃣ Si ME > 0 y PKT = 0 → 0 (empresa con margen pero sin productividad → no hay palanca real)
# 3️⃣ Si ME = 0 y PKT > 0 → 0 (empresa sin margen, pero con capital productivo no aprovechado)
# 4️⃣ Si ME > 0 y PKT > 0 → Normal (se calcula la eficiencia real del capital)

base_indicadores['PCD'] = np.where(
    (base_indicadores['ME'] == 0) & (base_indicadores['PKT'] == 0), np.nan,  # Caso 1: indeterminado
    np.where(
        (base_indicadores['PKT'] == 0), 0,  # Caso 2: sin productividad, sin eficiencia transformadora
        base_indicadores['ME'] / base_indicadores['PKT']  # Casos 3 y 4: cálculo normal
    )
)

# 🔹 **Resumen de los valores en PCD**
casos_normales_PCD = ((base_indicadores['PKT'] > 0)).sum()
casos_nan_PCD = ((base_indicadores['ME'] == 0) & (base_indicadores['PKT'] == 0)).sum()
casos_cero_por_productividad = ((base_indicadores['PKT'] == 0) & (base_indicadores['ME'] > 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en PCD (Palanca de Crecimiento):**")
print(f"✅ Casos normales (PKT > 0): {casos_normales_PCD}")
print(f"⚠️ Casos NaN (indeterminado: ME = 0 y PKT = 0): {casos_nan_PCD} → 📌 Se recomienda imputar con 0 en modelos predictivos.")
print(f"🔻 Casos con PCD = 0 por falta de productividad (PKT = 0 y ME > 0): {casos_cero_por_productividad} → 📌 Se interpreta como sin eficiencia transformadora.")


# 🔹 **Cálculo de EVA (Valor Económico Agregado)**
# 📌 Fórmula: EVA = EBITDA - (EBITDA * PCD)
# 📌 Interpretación:
#    - Representa la generación de valor económico después de considerar el costo del capital.
#    - Un valor positivo indica que la empresa está generando más retorno del requerido.
#    - Un valor negativo sugiere que la empresa no está cubriendo su costo de capital.
#
# 📌 Ajuste para modelos predictivos:
#    - Si PCD es NaN (empresa sin margen ni productividad), se asume como 0.
#    - Esto evita distorsiones y mantiene la coherencia del indicador.

# ✅ Reemplazar NaN de PCD por 0 SOLO para el cálculo de EVA (sin modificar el original)
PCD_ajustada = base_indicadores['PCD'].fillna(0)

# 🔧 Calcular EVA usando la versión ajustada
base_indicadores['EVA'] = base_indicadores['EBITDA'] - (base_indicadores['EBITDA'] * PCD_ajustada)

# 🔹 **Resumen de los valores en EVA**
casos_normales_EVA = (base_indicadores['EBITDA'] > 0).sum()
casos_cero_EVA = ((base_indicadores['EBITDA'] == 0) & (PCD_ajustada > 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en EVA (Valor Económico Agregado):**")
print(f"✅ Casos normales (EBITDA > 0): {casos_normales_EVA}")
print(f"🔘 Casos en cero (sin EBITDA pero con PCD): {casos_cero_EVA} → 📌 En modelos predictivos, se mantiene como 0 para evitar distorsiones.")



# ==============================================================================
# 📌 SECCIÓN 6: CÁLCULO DE Indicadores de Estructura Financiera
# ==============================================================================

# 🔹 **Cálculo de DCA (Deuda Total sobre Activos)**
# 📌 Fórmula: DCA = PT / AT
# 📌 Interpretación:
#    - Muestra qué porcentaje de los activos está financiado con deuda.
#    - Un valor alto indica una mayor dependencia del endeudamiento.
#
# 📌 Manejo de casos:
# 1️⃣ Si PT = 0 y AT = 0 → NaN (indeterminado, empresa sin deuda ni activos)
# 2️⃣ Si PT > 0 y AT = 0 → ∞ (empresa con deuda pero sin activos reportados)
# 3️⃣ Si PT = 0 y AT > 0 → 0 (empresa sin deuda, totalmente financiada por capital propio)
# 4️⃣ Si PT > 0 y AT > 0 → Normal (se calcula la división)

base_indicadores['DCA'] = np.where(
    (base_indicadores['PT'] == 0) & (base_indicadores['AT'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['AT'] == 0), np.inf,  # Caso 2: ∞ (cuando AT = 0)
        base_indicadores['PT'] / base_indicadores['AT']  # Caso 3 y 4 juntos
    )
)

# 🔹 **Resumen de los valores en DCA**
casos_normales_DCA = ((base_indicadores['AT'] > 0)).sum()  # Casos normales incluyen PT = 0
casos_nan_DCA = ((base_indicadores['PT'] == 0) & (base_indicadores['AT'] == 0)).sum()
casos_inf_DCA = ((base_indicadores['AT'] == 0) & (base_indicadores['PT'] > 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en DCA (Deuda Total sobre Activos):**")
print(f"✅ Casos normales: {casos_normales_DCA}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_DCA} → 📌 En modelos predictivos, **se recomienda imputar con 0**, ya que indica una empresa sin activos ni deuda.")
print(f"♾️ Casos Infinito positivo: {casos_inf_DCA} → 📌 Se recomienda **eliminar estos casos** en análisis financiero, ya que reflejan datos inconsistentes (deuda sin activos).")
# 🔹 **Cálculo de CPA (Capital Propio sobre Activos)**
# 📌 Fórmula: CPA = PN / AT
# 📌 Interpretación:
#    - Indica qué proporción de los activos está financiada con capital propio.
#    - Un valor alto significa mayor solidez financiera y menor dependencia del endeudamiento.
#
# 📌 Manejo de casos:
# 1️⃣ Si PN = 0 y AT = 0 → NaN (indeterminado, empresa sin capital ni activos)
# 2️⃣ Si PN > 0 y AT = 0 → ∞ (empresa con patrimonio pero sin activos reportados)
# 3️⃣ Si PN = 0 y AT > 0 → 0 (empresa sin capital propio, totalmente financiada con deuda)
# 4️⃣ Si PN > 0 y AT > 0 → Normal (se calcula la división)

base_indicadores['CPA'] = np.where(
    (base_indicadores['PN'] == 0) & (base_indicadores['AT'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['AT'] == 0), np.inf,  # Caso 2: ∞ (cuando AT = 0)
        base_indicadores['PN'] / base_indicadores['AT']  # Caso 3 y 4 juntos
    )
)

# 🔹 **Resumen de los valores en CPA**
casos_normales_CPA = ((base_indicadores['AT'] > 0)).sum()  # Casos normales incluyen PN = 0
casos_nan_CPA = ((base_indicadores['PN'] == 0) & (base_indicadores['AT'] == 0)).sum()
casos_inf_CPA = ((base_indicadores['AT'] == 0) & (base_indicadores['PN'] > 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en CPA (Capital Propio sobre Activos):**")
print(f"✅ Casos normales: {casos_normales_CPA}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_CPA} → 📌 En modelos predictivos, **se recomienda imputar con 0**, ya que representa una empresa sin activos ni patrimonio.")
print(f"♾️ Casos Infinito positivo: {casos_inf_CPA} → 📌 Se recomienda **eliminar estos casos** en análisis financiero, ya que indican inconsistencia en los datos (capital sin activos).")

# 🔹 **Cálculo de DATA (Deuda Total sobre Activos)**
# 📌 Fórmula: DATA = PT / AT
# 📌 Interpretación:
#    - Indica qué porcentaje de los activos de la empresa están financiados con deuda.
#    - Un valor alto sugiere una alta dependencia del financiamiento externo.
#    - Empresas con DATA elevado pueden enfrentar mayor riesgo financiero.
#
# 📌 Manejo de casos:
# 1️⃣ Si PT = 0 y AT = 0 → NaN (indeterminado, empresa sin activos ni pasivos).
# 2️⃣ Si PT > 0 y AT = 0 → ∞ (empresa con deuda pero sin activos reportados, lo cual no tiene sentido financiero).
# 3️⃣ Si PT = 0 y AT > 0 → 0 (empresa sin deuda, solo con activos propios).
# 4️⃣ Si PT > 0 y AT > 0 → Normal (se calcula la división).

base_indicadores['DATA'] = np.where(
    (base_indicadores['PT'] == 0) & (base_indicadores['AT'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['AT'] == 0), np.inf,  # Caso 2: ∞
        base_indicadores['PT'] / base_indicadores['AT']  # Caso 3 y 4 juntos
    )
)

# 🔹 **Resumen de los valores en DATA**
casos_normales_DATA = ((base_indicadores['AT'] > 0)).sum()
casos_nan_DATA = ((base_indicadores['PT'] == 0) & (base_indicadores['AT'] == 0)).sum()
casos_inf_DATA = ((base_indicadores['AT'] == 0) & (base_indicadores['PT'] > 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en DATA (Deuda Total sobre Activos):**")
print(f"✅ Casos normales: {casos_normales_DATA}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_DATA} → 📌 **Manejo recomendado:** **Imputar con 0** en modelos predictivos, ya que significa empresa sin activos ni deuda.")
print(f"♾️ Casos Infinito positivo: {casos_inf_DATA} → 📌 **Manejo recomendado:** **Eliminar estos casos**, pues una empresa con deuda pero sin activos no es financieramente consistente.")

# ==============================================================================

# 🔹 **Cálculo de DLPC (Deuda de Largo Plazo sobre Capital)**
# 📌 Fórmula: DLPC = PLP / PN
# 📌 Interpretación:
#    - Indica qué proporción del patrimonio de la empresa está comprometida con deuda a largo plazo.
#    - Un valor alto indica un mayor apalancamiento en el largo plazo.
#    - Empresas con DLPC alto pueden enfrentar dificultades de liquidez futura.
#
# 📌 Manejo de casos:
# 1️⃣ Si PLP = 0 y PN = 0 → NaN (indeterminado, empresa sin patrimonio ni pasivos de largo plazo).
# 2️⃣ Si PLP > 0 y PN = 0 → ∞ (empresa con deuda de largo plazo pero sin patrimonio, indicativo de alto riesgo financiero).
# 3️⃣ Si PLP = 0 y PN > 0 → 0 (empresa sin deuda de largo plazo, solo con patrimonio).
# 4️⃣ Si PLP > 0 y PN > 0 → Normal (se calcula la división).

base_indicadores['DLPC'] = np.where(
    (base_indicadores['PLP'] == 0) & (base_indicadores['PN'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['PN'] == 0), np.inf,  # Caso 2: ∞
        base_indicadores['PLP'] / base_indicadores['PN']  # Caso 3 y 4 juntos
    )
)

# 🔹 **Resumen de los valores en DLPC**
casos_normales_DLPC = ((base_indicadores['PN'] > 0)).sum()
casos_nan_DLPC = ((base_indicadores['PLP'] == 0) & (base_indicadores['PN'] == 0)).sum()
casos_inf_DLPC = ((base_indicadores['PN'] == 0) & (base_indicadores['PLP'] > 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en DLPC (Deuda de Largo Plazo sobre Capital):**")
print(f"✅ Casos normales: {casos_normales_DLPC}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_DLPC} → 📌 **Manejo recomendado:** **Imputar con 0**, ya que representa una empresa sin deuda de largo plazo ni patrimonio.")
print(f"♾️ Casos Infinito positivo: {casos_inf_DLPC} → 📌 **Manejo recomendado:** **Mantener en infinito o imputar con un valor alto (ej. 10)** para modelos predictivos.")


# 🔹 **Cálculo de CCNAT (Capital de Trabajo Neto sobre Activos Totales)**
# 📌 Fórmula: CCNAT = (AC - PCP) / AT
# 📌 Interpretación:
#    - Indica la proporción de los activos totales que representan el capital de trabajo neto.
#    - Un valor positivo sugiere que la empresa tiene más activos corrientes que pasivos corrientes, lo que es una señal de liquidez.
#    - Un valor negativo indica que la empresa depende demasiado del financiamiento de corto plazo.
#
# 📌 Manejo de casos:
# 1️⃣ Si (AC - PCP) = 0 y AT = 0 → NaN (indeterminado, empresa sin activos ni capital de trabajo).
# 2️⃣ Si (AC - PCP) > 0 y AT = 0 → ∞ (empresa con capital de trabajo positivo pero sin activos totales, inconsistente).
# 3️⃣ Si (AC - PCP) < 0 y AT = 0 → -∞ (empresa con capital de trabajo negativo pero sin activos totales, inconsistente).
# 4️⃣ Si (AC - PCP) = 0 y AT > 0 → 0 (empresa con capital de trabajo neto neutro).
# 5️⃣ Si (AC - PCP) > 0 y AT > 0 → Normal (se calcula la división).
# 6️⃣ Si (AC - PCP) < 0 y AT > 0 → Negativo (empresa con mayor pasivo corriente que activos corrientes).

base_indicadores['CCNAT'] = np.where(
    ((base_indicadores['AC'] - base_indicadores['PCP']) == 0) & (base_indicadores['AT'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['AT'] == 0) & ((base_indicadores['AC'] - base_indicadores['PCP']) > 0), np.inf,  # Caso 2: ∞
        np.where(
            (base_indicadores['AT'] == 0) & ((base_indicadores['AC'] - base_indicadores['PCP']) < 0), -np.inf,  # Caso 3: -∞
            (base_indicadores['AC'] - base_indicadores['PCP']) / base_indicadores['AT']  # Casos 4, 5 y 6 juntos
        )
    )
)

# 🔹 **Resumen de los valores en CCNAT**
casos_normales_CCNAT = ((base_indicadores['AT'] > 0)).sum()
casos_nan_CCNAT = (((base_indicadores['AC'] - base_indicadores['PCP']) == 0) & (base_indicadores['AT'] == 0)).sum()
casos_inf_CCNAT = ((base_indicadores['AT'] == 0) & ((base_indicadores['AC'] - base_indicadores['PCP']) > 0)).sum()
casos_neginf_CCNAT = ((base_indicadores['AT'] == 0) & ((base_indicadores['AC'] - base_indicadores['PCP']) < 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en CCNAT (Capital de Trabajo Neto sobre Activos Totales):**")
print(f"✅ Casos normales: {casos_normales_CCNAT}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_CCNAT} → 📌 **Manejo recomendado:** **Imputar con 0**, ya que indica empresa sin activos ni capital de trabajo.")
print(f"♾️ Casos Infinito positivo: {casos_inf_CCNAT} → 📌 **Manejo recomendado:** **Eliminar estos casos**, ya que son inconsistentes.")
print(f"♾️ Casos Infinito negativo: {casos_neginf_CCNAT} → 📌 **Manejo recomendado:** **Eliminar estos casos**, ya que son inconsistentes.")

# ==============================================================================

# 🔹 **Cálculo de CTNAT (Capital de Trabajo Neto sobre Activos Totales)**
# 📌 Fórmula: CTNAT = CTN / AT
# 📌 Interpretación:
#    - Similar a CCNAT, pero usa el capital de trabajo neto (CTN) previamente calculado.
#    - Muestra qué parte de los activos totales está comprometida con el capital de trabajo neto.
#    - Un valor positivo es señal de estabilidad financiera.
#    - Un valor negativo indica posible insolvencia a corto plazo.
#
# 📌 Manejo de casos:
# 1️⃣ Si CTN = 0 y AT = 0 → NaN (indeterminado, empresa sin activos ni capital de trabajo).
# 2️⃣ Si CTN > 0 y AT = 0 → ∞ (empresa con capital de trabajo positivo pero sin activos totales, inconsistente).
# 3️⃣ Si CTN < 0 y AT = 0 → -∞ (empresa con capital de trabajo negativo pero sin activos totales, inconsistente).
# 4️⃣ Si CTN = 0 y AT > 0 → 0 (empresa con capital de trabajo neutro).
# 5️⃣ Si CTN > 0 y AT > 0 → Normal (se calcula la división).
# 6️⃣ Si CTN < 0 y AT > 0 → Negativo (empresa con mayor pasivo corriente que activos corrientes).

base_indicadores['CTNAT'] = np.where(
    (base_indicadores['CTN'] == 0) & (base_indicadores['AT'] == 0), np.nan,  # Caso 1: NaN
    np.where(
        (base_indicadores['AT'] == 0) & (base_indicadores['CTN'] > 0), np.inf,  # Caso 2: ∞
        np.where(
            (base_indicadores['AT'] == 0) & (base_indicadores['CTN'] < 0), -np.inf,  # Caso 3: -∞
            base_indicadores['CTN'] / base_indicadores['AT']  # Casos 4, 5 y 6 juntos
        )
    )
)

# 🔹 **Resumen de los valores en CTNAT**
casos_normales_CTNAT = ((base_indicadores['AT'] > 0)).sum()
casos_nan_CTNAT = ((base_indicadores['CTN'] == 0) & (base_indicadores['AT'] == 0)).sum()
casos_inf_CTNAT = ((base_indicadores['AT'] == 0) & (base_indicadores['CTN'] > 0)).sum()
casos_neginf_CTNAT = ((base_indicadores['AT'] == 0) & (base_indicadores['CTN'] < 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en CTNAT (Capital de Trabajo Neto sobre Activos Totales):**")
print(f"✅ Casos normales: {casos_normales_CTNAT}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_CTNAT} → 📌 **Manejo recomendado:** **Imputar con 0**, ya que indica empresa sin activos ni capital de trabajo.")
print(f"♾️ Casos Infinito positivo: {casos_inf_CTNAT} → 📌 **Manejo recomendado:** **Eliminar estos casos**, ya que son inconsistentes.")
print(f"♾️ Casos Infinito negativo: {casos_neginf_CTNAT} → 📌 **Manejo recomendado:** **Eliminar estos casos**, ya que son inconsistentes.")

  # 🔹 **Cálculo de CCT (Capital Contable sobre Pasivos Totales)**
  # 📌 Fórmula: CCT = PN / (PN + PT)
  # 📌 Interpretación:
  #    - Indica la proporción del capital contable (patrimonio) en relación con la estructura de financiamiento total.
  #    - Un valor alto significa menor dependencia del endeudamiento y mayor estabilidad financiera.
  #
  # 📌 Manejo de casos:
  # 1️⃣ Si PN = 0 y PT = 0 → NaN (indeterminado, empresa sin patrimonio ni pasivos).
  # 2️⃣ Si PN > 0 y PT = 0 → 1 (empresa sin pasivos, totalmente financiada por capital propio).
  # 3️⃣ Si PN = 0 y PT > 0 → 0 (empresa sin capital propio, totalmente financiada con deuda).
  # 4️⃣ Si PN > 0 y PT > 0 → Normal (se calcula la división).

  base_indicadores['CCT'] = np.where(
      (base_indicadores['PN'] == 0) & (base_indicadores['PT'] == 0), np.nan,  # Caso 1: NaN
      np.where(
          (base_indicadores['PT'] == 0), 1,  # Caso 2: 1 (empresa sin pasivos)
          np.where(
              (base_indicadores['PN'] == 0), 0,  # Caso 3: 0 (empresa sin capital propio)
              base_indicadores['PN'] / (base_indicadores['PN'] + base_indicadores['PT'])  # Caso 4: Normal
          )
      )
  )

  # 🔹 **Resumen de los valores en CCT**
  casos_normales_CCT = ((base_indicadores['PN'] > 0) & (base_indicadores['PT'] > 0)).sum()
  casos_nan_CCT = ((base_indicadores['PN'] == 0) & (base_indicadores['PT'] == 0)).sum()
  casos_1_CCT = ((base_indicadores['PT'] == 0) & (base_indicadores['PN'] > 0)).sum()
  casos_0_CCT = ((base_indicadores['PN'] == 0) & (base_indicadores['PT'] > 0)).sum()

  # 📌 Imprimir los resultados para verificar
  print("\n📊 **Distribución de casos en CCT (Capital Contable sobre Pasivos Totales):**")
  print(f"✅ Casos normales: {casos_normales_CCT}")
  print(f"⚠️ Casos NaN (indeterminado): {casos_nan_CCT} → 📌 **Manejo recomendado:** **Imputar con 0.5**, para reflejar una estructura de financiamiento balanceada en ausencia de información.")
  print(f"🔹 Casos con valor 1: {casos_1_CCT} → 📌 Empresas sin pasivos, totalmente financiadas con patrimonio.")
  print(f"🔹 Casos con valor 0: {casos_0_CCT} → 📌 Empresas sin capital propio, financiadas únicamente con deuda.")

# ==============================================================================

# 🔹 **Cálculo de CCNCT (Capital de Trabajo Neto sobre Capital Total)**
# 📌 Fórmula: CCNCT = (AC - PCP) / (PN + PT)
# 📌 Interpretación:
#    - Mide la proporción del capital total que está comprometida con el capital de trabajo neto.
#    - Un valor positivo indica mayor liquidez y capacidad de cubrir pasivos a corto plazo con activos corrientes.
#    - Un valor negativo indica que la empresa depende demasiado del financiamiento externo a corto plazo.
#
# 📌 Manejo de casos:
# 1️⃣ Si (AC - PCP) = 0 y (PN + PT) = 0 → NaN (indeterminado, empresa sin capital total ni capital de trabajo).
# 2️⃣ Si (AC - PCP) > 0 y (PN + PT) = 0 → ∞ (empresa con capital de trabajo positivo pero sin capital total, inconsistente).
# 3️⃣ Si (AC - PCP) < 0 y (PN + PT) = 0 → -∞ (empresa con capital de trabajo negativo pero sin capital total, inconsistente).
# 4️⃣ Si (AC - PCP) = 0 y (PN + PT) > 0 → 0 (empresa con capital de trabajo neutro).
# 5️⃣ Si (AC - PCP) > 0 y (PN + PT) > 0 → Normal (se calcula la división).
# 6️⃣ Si (AC - PCP) < 0 y (PN + PT) > 0 → Negativo (empresa con mayor pasivo corriente que activos corrientes).

base_indicadores['CCNCT'] = np.where(
    ((base_indicadores['AC'] - base_indicadores['PCP']) == 0) & ((base_indicadores['PN'] + base_indicadores['PT']) == 0), np.nan,  # Caso 1: NaN
    np.where(
        ((base_indicadores['PN'] + base_indicadores['PT']) == 0) & ((base_indicadores['AC'] - base_indicadores['PCP']) > 0), np.inf,  # Caso 2: ∞
        np.where(
            ((base_indicadores['PN'] + base_indicadores['PT']) == 0) & ((base_indicadores['AC'] - base_indicadores['PCP']) < 0), -np.inf,  # Caso 3: -∞
            (base_indicadores['AC'] - base_indicadores['PCP']) / (base_indicadores['PN'] + base_indicadores['PT'])  # Casos 4, 5 y 6 juntos
        )
    )
)

# 🔹 **Resumen de los valores en CCNCT**
casos_normales_CCNCT = ((base_indicadores['PN'] + base_indicadores['PT'] > 0)).sum()
casos_nan_CCNCT = (((base_indicadores['AC'] - base_indicadores['PCP']) == 0) & ((base_indicadores['PN'] + base_indicadores['PT']) == 0)).sum()
casos_inf_CCNCT = (((base_indicadores['PN'] + base_indicadores['PT']) == 0) & ((base_indicadores['AC'] - base_indicadores['PCP']) > 0)).sum()
casos_neginf_CCNCT = (((base_indicadores['PN'] + base_indicadores['PT']) == 0) & ((base_indicadores['AC'] - base_indicadores['PCP']) < 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en CCNCT (Capital de Trabajo Neto sobre Capital Total):**")
print(f"✅ Casos normales: {casos_normales_CCNCT}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_CCNCT} → 📌 **Manejo recomendado:** **Imputar con 0**, ya que representa ausencia de capital total y capital de trabajo neto.")
print(f"♾️ Casos Infinito positivo: {casos_inf_CCNCT} → 📌 **Manejo recomendado:** **Eliminar estos casos**, ya que son inconsistentes.")
print(f"♾️ Casos Infinito negativo: {casos_neginf_CCNCT} → 📌 **Manejo recomendado:** **Eliminar estos casos**, ya que son inconsistentes.")

# 🔹 **Cálculo de CTNCT (Capital de Trabajo Neto sobre Capital Total)**
# 📌 Fórmula: CTNCT = (AC - PCP) / (PN + PT)
# 📌 Interpretación:
#    - Muestra qué proporción del capital total está destinada al capital de trabajo neto.
#    - Un valor alto indica mayor capacidad de la empresa para operar sin depender del financiamiento externo a corto plazo.
#    - Un valor negativo sugiere que los pasivos corrientes superan los activos corrientes, lo que indica problemas de liquidez.
#
# 📌 Manejo de casos:
# 1️⃣ Si (AC - PCP) = 0 y (PN + PT) = 0 → NaN (indeterminado, empresa sin capital total ni capital de trabajo).
# 2️⃣ Si (AC - PCP) > 0 y (PN + PT) = 0 → ∞ (empresa con capital de trabajo pero sin capital total, inconsistente).
# 3️⃣ Si (AC - PCP) < 0 y (PN + PT) = 0 → -∞ (empresa con capital de trabajo negativo pero sin capital total, inconsistente).
# 4️⃣ Si (AC - PCP) = 0 y (PN + PT) > 0 → 0 (empresa con capital de trabajo neutro).
# 5️⃣ Si (AC - PCP) > 0 y (PN + PT) > 0 → Normal (se calcula la división).
# 6️⃣ Si (AC - PCP) < 0 y (PN + PT) > 0 → Negativo (empresa con pasivos corrientes superiores a activos corrientes).

base_indicadores['CTNCT'] = np.where(
    ((base_indicadores['AC'] - base_indicadores['PCP']) == 0) & ((base_indicadores['PN'] + base_indicadores['PT']) == 0), np.nan,  # Caso 1: NaN
    np.where(
        ((base_indicadores['PN'] + base_indicadores['PT']) == 0) & ((base_indicadores['AC'] - base_indicadores['PCP']) > 0), np.inf,  # Caso 2: ∞
        np.where(
            ((base_indicadores['PN'] + base_indicadores['PT']) == 0) & ((base_indicadores['AC'] - base_indicadores['PCP']) < 0), -np.inf,  # Caso 3: -∞
            (base_indicadores['AC'] - base_indicadores['PCP']) / (base_indicadores['PN'] + base_indicadores['PT'])  # Casos 4, 5 y 6 juntos
        )
    )
)

# 🔹 **Resumen de los valores en CTNCT**
casos_normales_CTNCT = ((base_indicadores['PN'] + base_indicadores['PT'] > 0)).sum()
casos_nan_CTNCT = (((base_indicadores['AC'] - base_indicadores['PCP']) == 0) & ((base_indicadores['PN'] + base_indicadores['PT']) == 0)).sum()
casos_inf_CTNCT = (((base_indicadores['PN'] + base_indicadores['PT']) == 0) & ((base_indicadores['AC'] - base_indicadores['PCP']) > 0)).sum()
casos_neginf_CTNCT = (((base_indicadores['PN'] + base_indicadores['PT']) == 0) & ((base_indicadores['AC'] - base_indicadores['PCP']) < 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en CTNCT (Capital de Trabajo Neto sobre Capital Total):**")
print(f"✅ Casos normales: {casos_normales_CTNCT}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_CTNCT} → 📌 **Manejo recomendado:** **Imputar con 0**, ya que representa ausencia de capital total y capital de trabajo neto.")
print(f"♾️ Casos Infinito positivo: {casos_inf_CTNCT} → 📌 **Manejo recomendado:** **Eliminar estos casos**, ya que son inconsistentes.")
print(f"♾️ Casos Infinito negativo: {casos_neginf_CTNCT} → 📌 **Manejo recomendado:** **Eliminar estos casos**, ya que son inconsistentes.")

# ==============================================================================

# 🔹 **Cálculo de AFT (Activos Fijos sobre Total Financiamiento)**
# 📌 Fórmula: AFT = (AF + PT) / (AF + PN)
# 📌 Interpretación:
#    - Mide qué proporción del total de activos fijos está financiada por pasivos en lugar de patrimonio.
#    - Un valor mayor a 1 sugiere que los activos fijos dependen más del endeudamiento que del capital propio.
#    - Un valor menor a 1 indica que la empresa financia sus activos fijos mayormente con su patrimonio.
#
# 📌 Manejo de casos:
# 1️⃣ Si (AF + PT) = 0 y (AF + PN) = 0 → NaN (indeterminado, empresa sin activos fijos ni financiamiento).
# 2️⃣ Si (AF + PN) = 0 → ∞ (empresa con activos fijos pero sin patrimonio ni financiamiento, inconsistente).
# 3️⃣ Si (AF + PT) = 0 → 0 (empresa sin pasivos ni activos fijos, solo con patrimonio).
# 4️⃣ Si (AF + PT) > 0 y (AF + PN) > 0 → Normal (se calcula la división).

base_indicadores['AFT'] = np.where(
    ((base_indicadores['AF'] + base_indicadores['PT']) == 0) & ((base_indicadores['AF'] + base_indicadores['PN']) == 0), np.nan,  # Caso 1: NaN
    np.where(
        ((base_indicadores['AF'] + base_indicadores['PN']) == 0), np.inf,  # Caso 2: ∞
        np.where(
            ((base_indicadores['AF'] + base_indicadores['PT']) == 0), 0,  # Caso 3: 0
            (base_indicadores['AF'] + base_indicadores['PT']) / (base_indicadores['AF'] + base_indicadores['PN'])  # Caso 4: Normal
        )
    )
)

# 🔹 **Resumen de los valores en AFT**
casos_normales_AFT = ((base_indicadores['AF'] + base_indicadores['PN'] > 0)).sum()
casos_nan_AFT = (((base_indicadores['AF'] + base_indicadores['PT']) == 0) & ((base_indicadores['AF'] + base_indicadores['PN']) == 0)).sum()
casos_inf_AFT = (((base_indicadores['AF'] + base_indicadores['PN']) == 0)).sum()
casos_cero_AFT = (((base_indicadores['AF'] + base_indicadores['PT']) == 0)).sum()

# 📌 Imprimir los resultados para verificar
print("\n📊 **Distribución de casos en AFT (Activos Fijos sobre Total Financiamiento):**")
print(f"✅ Casos normales: {casos_normales_AFT}")
print(f"⚠️ Casos NaN (indeterminado): {casos_nan_AFT} → 📌 **Manejo recomendado:** **Imputar con 0**, ya que representa una empresa sin activos fijos ni financiamiento.")
print(f"♾️ Casos Infinito positivo: {casos_inf_AFT} → 📌 **Manejo recomendado:** **Eliminar estos casos**, ya que son inconsistentes.")
print(f"🔹 Casos con valor 0: {casos_cero_AFT} → 📌 Empresa sin pasivos ni activos fijos, financiada únicamente con patrimonio.")




# ==============================================================================
# 📌 RESUMEN GENERAL DE INDICADORES CALCULADOS
# ==============================================================================

# 🔹 Obtener lista de columnas nuevas (indicadores calculados)
indicadores_calculados = base_indicadores.columns.difference(base_datos.columns).tolist()

print(f"\n📊 🔎 Se han calculado {len(indicadores_calculados)} indicadores financieros.")
print("📋 Lista de indicadores calculados:")
print(indicadores_calculados)

# 🔹 Crear resumen de NaN, inf, valores válidos por cada indicador
resumen_indicadores = []

for col in indicadores_calculados:
    total = len(base_indicadores)
    nan_count = base_indicadores[col].isna().sum()
    inf_count = np.isinf(base_indicadores[col]).sum()
    validos = total - nan_count - inf_count
    resumen_indicadores.append({
        'Indicador': col,
        'Total': total,
        'NaN': nan_count,
        '∞': inf_count,
        'Válidos': validos
    })

# 🔹 Mostrar el resumen en tabla ordenada
df_resumen = pd.DataFrame(resumen_indicadores).sort_values(by='Indicador')
print("\n📊 **Resumen general de calidad de datos por indicador:**")
import pandas as pd
from IPython.display import display
display(df_resumen)


# ====================================================
# 📌 Imputación de valores NaN según recomendaciones
# ====================================================


# 🔹 Diccionarios con el valor recomendado de imputación para cada indicador
imputar_con_1 = {
    'LC', 'PA', 'LG', 'T', 'TN', 'RAO', 'RCD', 'RCID', 'RCCP', 'RCLP', 'RCDT', 'PKT'
}

imputar_con_0 = {
    'RET', 'RECP', 'RELP', 'RAF', 'EBITDA', 'MB', 'MO', 'ROA', 'ROE', 'RSV', 'ROI',
    'ME', 'RA', 'RI', 'RCC', 'RCP', 'PPC', 'PPP', 'PPI', 'KTNO', 'DCA', 'CPA', 'DATA',
    'DLPC', 'CCNAT', 'CTNAT', 'CCNCT', 'CTNCT', 'AFT', 'PCD'
}

imputar_con_05 = {
    'CCT'  # Se propone 0.5 como valor neutral para balance patrimonio/pasivo
}

# 🔹 Aplicar imputaciones al DataFrame de indicadores
for col in base_indicadores.columns:
    if col in imputar_con_1:
        base_indicadores[col] = base_indicadores[col].fillna(1)
    elif col in imputar_con_0:
        base_indicadores[col] = base_indicadores[col].fillna(0)
    elif col in imputar_con_05:
        base_indicadores[col] = base_indicadores[col].fillna(0.5)

# 📌 Nota:
# Esta imputación es útil para modelos predictivos.
# Si el análisis fuera estrictamente financiero, puede ser preferible mantener los NaN.

# 📊 RESUMEN ACTUALIZADO DE CALIDAD DE DATOS POR INDICADOR (CON ∞ POSITIVO Y NEGATIVO DISTINTOS)

resumen_post_imputacion = []

for col in indicadores_calculados:
    total = len(base_indicadores)
    nan_count = base_indicadores[col].isna().sum()
    pos_inf = np.isposinf(base_indicadores[col]).sum()   # ∞ positivo
    neg_inf = np.isneginf(base_indicadores[col]).sum()   # ∞ negativo
    validos = total - nan_count - pos_inf - neg_inf

    resumen_post_imputacion.append({
        'Indicador': col,
        'Total': total,
        'NaN': nan_count,
        '+∞': pos_inf,
        '-∞': neg_inf,
        'Válidos': validos
    })

# Crear DataFrame ordenado
df_resumen_post = pd.DataFrame(resumen_post_imputacion).sort_values(by='Indicador').reset_index(drop=True)
# Mostrar el resumen actualizado
print("\n📊 **Resumen actualizado de calidad de datos por indicador después de las imputaciones sugeridas (con ∞ positivo y negativo separados):**")
from IPython.display import display
display(df_resumen_post)




# 📌 Crear base para empresas turísticas
# Usamos la base con columna Turismo antes de eliminarla
columnas_cuentas = [
    'AC', 'I', 'AF', 'AT', 'PN', 'PLP', 'PCP', 'PT', 'CPC', 'CPP',
    'UN', 'V', 'UB', 'EBIT', 'UO', 'GF', 'GV', 'GA', 'Imp', 'DyA', 'FCL'
]
columnas_ciiu_quitar = ['CIIU', 'CIIU_Codigo', 'CIIU_Sector', 'Ciudad']
columnas_a_quitar = columnas_cuentas + columnas_ciiu_quitar

base_modelos_turismo = base_indicadores[base_indicadores["Turismo"] == 1].copy()
base_modelos_turismo = base_modelos_turismo.drop(columns=columnas_a_quitar + ['Turismo'])  # Quitamos todo

# 📌 Guardar solo la base de turismo
ruta_final = "/content/drive/MyDrive/Datos/6_Base_Modelos_Predictivos.parquet"
base_modelos_turismo.to_parquet(ruta_final, index=False)
print(f"✅ Base final para modelos del sector turístico guardada en:\n{ruta_final}")

# 📄 Crear resumen .txt del paso
resumen_txt = f"""
📄 Script: 6_Creacion_Indicadores_Financieros.ipynb
📅 Fecha: {pd.Timestamp.today().strftime('%Y-%m-%d')}
🧩 Objetivo: Calcular indicadores financieros a partir de cuentas imputadas y construir la base final para modelos del sector turismo.

🔁 Pasos realizados:
- Cálculo detallado de más de 50 indicadores financieros.
- Revisión y tratamiento explícito de NaNs, ∞ y casos especiales.
- Imputación guiada por criterios contables para uso en modelos.
- Eliminación de columnas contables y variables administrativas.
- Filtrado de registros con Turismo = 1.
- Construcción final de base para modelos predictivos.

📦 Archivo generado:
- 6_Base_Modelos_Predictivos.parquet

📊 Estadísticas:
- Registros finales: {len(base_modelos_turismo):,}
- NITs únicos: {base_modelos_turismo['NIT'].nunique():,}
- Columnas finales: {len(base_modelos_turismo.columns)}

📋 Lista de columnas:
{chr(10).join(['   • ' + col for col in base_modelos_turismo.columns])}
"""

ruta_resumen = "/content/drive/MyDrive/Datos/6_resumen.txt"
with open(ruta_resumen, "w", encoding="utf-8") as f:
    f.write(resumen_txt.strip())

print(f"\n📝 Resumen guardado en: {ruta_resumen}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🔎 Información general de la base de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539318 entries, 0 to 539317
Data columns (total 31 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   NIT          539318 non-null  float64
 1   Año          539318 non-null  int64  
 2   Ciudad       539318 non-null  object 
 3   CIIU         539318 non-null  object 
 4   DEP          539318 non-null  object 
 5   AC           539318 non-null  float64
 6   I            539318 non-null  float64
 7   AF           539318 non-null  float64
 8   AT           539318 non-null  float64
 9   PN           539318 non-null  float64
 10  PLP          539318 non-null  float64
 11  PCP          539318 non-null  float64
 12  PT           539318 non-null  float64
 13  CPC          539318 non-null  float64
 14  CPP          539318 non-

,Indicador,Total,NaN,∞,Válidos
0,AFT,539318,18,19,539281
1,CCNAT,539318,21,13,539284
2,CCNCT,539318,21,14,539283
3,CCT,539318,20,15,539283
4,CPA,539318,19,15,539284
5,CTN,539318,0,0,539318
6,CTNAT,539318,21,13,539284
7,CTNCT,539318,21,14,539283
8,DATA,539318,19,15,539284
9,DCA,539318,19,15,539284



📊 **Resumen actualizado de calidad de datos por indicador después de las imputaciones sugeridas (con ∞ positivo y negativo separados):**


,Indicador,Total,NaN,+∞,-∞,Válidos
0,AFT,539318,0,19,0,539299
1,CCNAT,539318,0,1,12,539305
2,CCNCT,539318,0,2,12,539304
3,CCT,539318,0,0,15,539303
4,CPA,539318,0,15,0,539303
5,CTN,539318,0,0,0,539318
6,CTNAT,539318,0,1,12,539305
7,CTNCT,539318,0,2,12,539304
8,DATA,539318,0,15,0,539303
9,DCA,539318,0,15,0,539303


✅ Base final para modelos del sector turístico guardada en:
/content/drive/MyDrive/Datos/6_Base_Modelos_Predictivos.parquet

📝 Resumen guardado en: /content/drive/MyDrive/Datos/6_resumen.txt
